In [1]:
1+14

15

In [11]:
import numpy as np 
import awkward as ak 
import pandas as pd 
#import dask
#import dask.dataframe as dd

# I/O 
import uproot as ur
import hipopy.hipopy as hp # <--- Package for reading in the hipo files   

# Plotting
import matplotlib.pyplot as plt 

# Physics
from particle import PDGID
 
# Miscellaneous 
import os
import sys #NOTE: ADDED
import tqdm

# ML Imports
import torch
from torch import nn 
from torch.utils.data import Dataset, DataLoader, random_split
import torch_geometric as tg
# HIPO bank reading and linking functions

from torch_geometric.data import Data as dt
 
def get_bank_keys(bank_name,all_keys,separator='_'):
    """
    :description: Get list of the keys for given bank name from a list of all batch keys.
    
    :param: bank_name
    :param: all_keys
    :param: separator='_'
    
    :return: bank_keys
    """
    bank_keys = []
    for key in all_keys:
        if key.startswith(bank_name+separator):
            bank_keys.append(key)
    return bank_keys
        
def get_event_table(bank_keys,event_num,batch,dtype=float): 
    """
    :description: Get a bank event table as a numpy array of shape (number of columns, number of rows).
    
    :param: bank_keys 
    :param: event_num
    :param: batch
    :param: dtype=float
    
    :return: bank_table
    """
    bank_table = []
    bank_table = np.moveaxis(np.array([batch[key][event_num] for key in bank_keys], dtype=dtype),[0,1],[1,0])
    return bank_table

def get_link_indices(event_table_rec_particle,event_table,pindex_idx=1):
    """
    :description: Get index pairs linking entries in a bank back to entries in the 'REC::Particle' bank.
    
    :param: event_table_rec_particle 
    :param: event_table
    :param: pindex_idx=1
    
    :return: link_indices
    """
    
    link_indices = []
    nrec = np.shape(event_table_rec_particle)[0]
    for rec_particle_idx in range(0,nrec):
        for event_table_idx, el in enumerate(event_table[:,pindex_idx]):
            if el==rec_particle_idx:
                link_indices.append([rec_particle_idx,event_table_idx])
    return np.array(link_indices,dtype=int) #NOTE: link_indices = [(event_table_idx,rec_particle_idx)]

def get_parent_indices(mc_event_table,index_idx=0,parent_idx=4,daughter_idx=5):
    """
    TODO
    """
    for mc_event_table_idx, index in enumerate(mc_event_table[:,index_idx]):
        pass
    pass
#get_match_
def get_match_indices(
    rec_event_table,
    mc_event_table,
    rec_px_idx             = 1,
    rec_py_idx             = 2,
    rec_pz_idx             = 3,
    rec_ch_idx             = 8,
    mc_px_idx              = 6,
    mc_py_idx              = 7, 
    mc_pz_idx              = 8,
    mc_pid_idx             = 3,
    mc_daughter_idx        = 5,
    match_charge           = True,
    require_no_mc_daughter = True,
    enforce_uniqueness     = True,
    ):
    
    # Set minimum
    rec_final_state_min_idx = 1
    mc_final_state_min_idx  = 3 #NOTE: MC::Lund bank is structured [e, p, q, e', all the other final state particles...]
    
    # Initialize index map
    match_indices    = -np.ones((rec_event_table.shape[0],2),dtype=float)
    match_indices[0] = [0,3] #NOTE: Always match first entry in REC::Particle to scattered electron in MC::Lund.

    # Get REC::Particle info
    rec_px    = rec_event_table[:,rec_px_idx]
    rec_py    = rec_event_table[:,rec_py_idx]
    rec_pz    = rec_event_table[:,rec_pz_idx]
    rec_pT    = np.sqrt(np.square(rec_event_table[:,rec_px_idx])+np.square(rec_event_table[:,rec_py_idx]))
    rec_p     = np.sqrt(np.square(rec_event_table[:,rec_px_idx])+np.square(rec_event_table[:,rec_py_idx])+np.square(rec_event_table[:,rec_pz_idx]))
    rec_theta = np.array(rec_pz)
    rec_theta = np.arctan(rec_pT,rec_theta)
    rec_phi   = np.arctan2(rec_py,rec_px)
    
    # Get MC::Lund info
    mc_px    = mc_event_table[:,mc_px_idx]
    mc_py    = mc_event_table[:,mc_py_idx]
    mc_pz    = mc_event_table[:,mc_pz_idx]
    mc_pT    = np.sqrt(np.square(mc_event_table[:,mc_px_idx])+np.square(mc_event_table[:,mc_py_idx]))
    mc_p     = np.sqrt(np.square(mc_event_table[:,mc_px_idx])+np.square(mc_event_table[:,mc_py_idx])+np.square(mc_event_table[:,mc_pz_idx]))
    mc_theta = np.array(mc_pz)
    mc_theta = np.arctan(mc_pT,mc_theta)
    mc_phi   = np.arctan2(mc_py,mc_px)

    # Loop rec particles
    for rec_idx, rec_part in enumerate(rec_event_table):
        
        # Start with final state particles past scattered electron
        if rec_idx<rec_final_state_min_idx: continue
        
        # Get REC::Particle charge
        rec_ch = rec_event_table[rec_idx,rec_ch_idx]
        
        # Loop mc particles
        mc_match_idx = -1
        min_domega   = 9999
        for mc_idx, mc_part in enumerate(mc_event_table):
            
            # Start with final state particles past scattered electron
            if mc_idx<mc_final_state_min_idx:
                continue
            
            # Enforce unique matching
            if enforce_uniqueness and mc_idx in match_indices[:,1]:
                continue
            
            # Match charge and require that the MC particle be final state (no daughters)
            if match_charge and rec_ch!=PDGID(mc_event_table[mc_idx,mc_pid_idx]).charge:
                continue
            if require_no_mc_daughter and mc_event_table[mc_idx,mc_daughter_idx]!=0:
                continue
                
            # Get angular and momentum differences
            dp     = np.abs(rec_p[rec_idx]     - mc_p[mc_idx])
            dtheta = np.abs(rec_theta[rec_idx] - mc_theta[mc_idx])
            dphi   = np.abs(rec_phi[rec_idx]   - mc_phi[mc_idx]) if np.abs(rec_phi[rec_idx] - mc_phi[mc_idx])<np.pi else 2*np.pi-np.abs(rec_phi[rec_idx] - mc_phi[mc_idx])
            domega = dp**2 + dtheta**2 + dphi**2
            
            # Reset angular, momentum minimum difference
            if domega<min_domega:
                min_domega   = domega
                mc_match_idx = mc_idx
                
        # Append matched index pair
        match_indices[rec_idx] = [rec_idx,mc_match_idx]
        
    return np.array(match_indices,dtype=int) #NOTE: IMPORTANT!

def get_info(base_indices,link_indices,bank_entry_indices,bank_event_table):
    """
    :description: Get selected entry info from other banks linked to REC::Particle.
    
    :param: base_indices
    :param: link_indices #NOTE: if None assume bank is REC::Particle and use identity map
    :param: bank_entry_indices
    :param: bank_event_table
    
    :return: bank_info as awkward.Array
    """
    if link_indices is None:
        bank_info = []
        for base_idx in base_indices:
            base_info = bank_event_table[base_idx,bank_entry_indices]
            bank_info.append([base_info])
            
        return ak.Array(bank_info)
            
    bank_info = []
    for base_idx in base_indices:
        base_info = []
        for rec_particle_idx, link_idx in link_indices:
            if rec_particle_idx==base_idx:
                base_info.append(bank_event_table[link_idx,bank_entry_indices]) #NOTE: INDICES HAVE TO BE INTEGERS...COULD ADD CHECK...
        if len(base_info)==0: #NOTE: Address case that no matches exist between banks
            base_info.append(np.zeros((len(bank_entry_indices),)))
        bank_info.append(base_info)
    
    return ak.Array(bank_info)

def get_truth_info(base_indices,match_indices,truth_entry_indices,mc_event_table):
    """
    :description: Get selected entry info from other banks linked to REC::Particle.
    
    :param: base_indices
    :param: link_indices #NOTE: if None assume bank is REC::Particle and use identity map
    :param: bank_entry_indices
    :param: bank_event_table
    
    :return: bank_info as awkward.Array
    """
    
    bank_info = []
    for base_idx in base_indices:
        base_info = []
        for rec_particle_idx, match_idx in match_indices:
            if rec_particle_idx==base_idx:
                base_info.append(mc_event_table[match_idx,truth_entry_indices]) #NOTE: INDICES HAVE TO BE INTEGERS...COULD ADD CHECK...
        if len(base_info)==0: #NOTE: Address case that no matches exist between banks
            base_info.append(np.zeros((len(truth_entry_indices),)))
        bank_info.append(base_info)
    
    return ak.Array(bank_info)

def check_has_decay(rec_particle_event_table,mc_lund_event_table,match_indices,decay,
                    rec_particle_pid_idx=0,mc_lund_pid_idx=3,mc_lund_parent_idx=4,mc_lund_daughter_idx=5):
    
    """
    :description: Check if specified decay is present in MC::Lund bank and the decay daughters are
        matched to particles of the same pid in REC::Particle
    
    :param: rec_particle_event_table
    :param: mc_lund_event_table
    :param: match_indices
    :param: decay
    :param: rec_particle_pid_idx = 0
    :param: mc_lund_pid_idx      = 3
    :param: mc_lund_parent_idx   = 4
    :param: mc_lund_daughter_idx = 5
    
    :return: boolean has_decay
    """
    
    has_decay = False
    decay_indices = None
    rec_indices = [-1 for i in range(len(decay[1]))] #NOTE: This assumes a 1-level decay...#TODO: Write more robust algorithm.
    if np.max(match_indices[:,-1])==-1: return has_decay, rec_indices #NOTE: This is the case of no matches at all.
    
    # Check if parent pid is in MC::Lund
    if not decay[0] in mc_lund_event_table[:,mc_lund_pid_idx]: return has_decay, rec_indices
    
    # Loop MC::Lund to find parent
    for parent_idx, parent_pid in enumerate(mc_lund_event_table[:,mc_lund_pid_idx]):
        if parent_pid==decay[0]:
#             print("DEBUGGING: found parent_pid == ",decay[0])
            
            # Check daughter pids in MC::Lund
            daughter_idx     = int(mc_lund_event_table[parent_idx,mc_lund_daughter_idx])-1 #NOTE: -1 is important because Lund index begins at 1.
            try:
                daughter_pids    = mc_lund_event_table[daughter_idx:daughter_idx+len(decay[1]),mc_lund_pid_idx]
            except Exception as e:
                print(e)
                print("DEBUGGING: np.shape(mc_lund_event_table) = ",np.shape(mc_lund_event_table))
                print("DEBUGGING: daughter_idx        = ",daughter_idx)
                print("DEBUGGING: decay = ",decay)
                raise Exception
            daughter_parents = mc_lund_event_table[daughter_idx:daughter_idx+len(decay[1]),mc_lund_parent_idx]
            daughter_parents -= 1 #NOTE: Important because Lund index begins at 1.
            if np.all(daughter_parents==parent_idx) and np.all(daughter_pids==decay[1]): #NOTE: this relies on input arrays being np...
                decay_indices = [parent_idx,[daughter_idx+i for i in range(len(decay[1]))]]
                rec_indices = [-1 for i in range(len(decay[1]))]#NOTE: Reset in case you have two not fully matched decays...unlikely but possible.
                
                # Now check that there is a match of the same pid in REC::Particle
                num_matched_daughters = 0
                num_daughters         = len(decay[1])
                for decay_idx, mc_idx in enumerate(decay_indices[1]):
                    for el in match_indices:
                        if el[1]==mc_idx:
                            rec_idx = el[0]
                            if rec_particle_event_table[rec_idx,rec_particle_pid_idx] == decay[1][decay_idx]:
                                num_matched_daughters += 1
                                rec_indices[decay_idx] = rec_idx #NOTE: That this is the actual index beginning at 0.
#                 print("DEBUGGING: num_matched_daughters = ",num_matched_daughters)
#                 print("DEBUGGING: decay_indices = ",decay_indices)
                if num_matched_daughters == num_daughters:
                    has_decay = True
    
    return has_decay, rec_indices 

# Select requested rows from REC::Particle data
def get_sub_array(arr,indices):
    
    """
    :description: Get sub array at indices along axis 1.
    
    :param: arr
    :param: indices
    
    :return: np.array new_arr
    """
    
    new_array = []
    for i in indices:
        new_array.append(arr[:,i])
    new_array = np.moveaxis(np.array(new_array),[0,1],[1,0])

    return new_array

def replace_pids(arr,pid_map,pid_i=0):
    """
    :description: Replace pids in given array roughly following scheme described in arxiv:1810.05165.
    
    :param: arr masked ndarray with dtype=float
    :param: pid_i last depth index for pids in arr
    """
    
    if 'int' in str(arr.dtype):
        print(" *** ERROR *** array passed to replace_pids should not have dtype==int")
        return
    
    for key in pid_map:
        arr[:,pid_i] = np.where(arr[:,pid_i]==key,
                                  pid_map[key],
                                  arr[:,pid_i])

def preprocess(x):
    """
    :description: Run preprocessing on input array.
    
    :param: x
    
    :return: new_x
    """
    #NOTE: Assume indices go 0-6 : pid, px, py, pz, beta, chi2pid, status
    
    pid_map = {
        22:0.0,
        11:-1.0,
        -11:1.0,
        2212:0.8,
        -2212:-0.8,
        2112:0.5,
        111:0.1,
        211:0.6,
        -211:-0.6,
        311:0.3,
        321:0.4,
        -321:-0.4,
        45:0.0
    }
    
    pid_idx, px_idx, py_idx, pz_idx, beta_idx, chi2pid_idx, status_idx = [i for i in range(np.shape(x)[-1])]
    
    # Reassign PIDs
    replace_pids(x,pid_map,pid_i=0)
    
    # Compute new arrays
    pT    = np.sqrt(np.add(np.square(x[:,px_idx]),np.square(x[:,py_idx])))
    phi   = np.arctan2(x[:,py_idx],x[:,px_idx])
    theta = np.divide(pT,x[:,pz_idx])
    beta  = np.log(x[:,beta_idx])
    
    #TODO: Compute event level differences and normalize: pT, phi, theta, beta (apply lognorm first though...)
    pT    -= pT.mean()
    pT    /= np.abs(pT).max() if np.abs(pT).max() > 0.0 else 1.0
    phi   -= phi.mean()
    phi   /= np.abs(phi).max() if np.abs(phi).max() > 0.0 else 1.0
    theta -= theta.mean()
    theta /= np.abs(theta).max() if np.abs(theta).max() > 0.0 else 1.0
    beta  -= beta.mean()
    beta  /= np.abs(beta).max() if np.abs(beta).max() > 0.0 else 1.0
    
    # Reset px, py, pz, beta -> pT, phi, theta, beta
    x[:,px_idx]   = pT
    x[:,py_idx]   = phi
    x[:,pz_idx]   = theta
    x[:,beta_idx] = beta
    
#     # Preprocess pT, phi, theta
#     x[:,px_idx] /= 10.0
#     x[:,py_idx] /= np.pi
#     x[:,pz_idx] /= np.pi
    
#     # Preprocess beta variable
#     x[:,beta_idx] -= 1
    
    # Preprocess chi2pid
    chi2_max, chi2_default, chi2_replacement   = 10, 9999, 10 #NOTE: DEBUGGING: USED TO BE >=chi2_default below!
    x[:,chi2pid_idx]  = np.where(x[:,chi2pid_idx]>=chi2_default, chi2_replacement, x[:,chi2pid_idx])
    x[:,chi2pid_idx] /= chi2_max #NOTE: IMPORTANT!  NEEDS TO BE AFTER REPLACEMENT OF MAX ABOVE!
    
    # Preprocess status variable
    x[:,status_idx] /= 5000
    
    # Reassign all data values that are NaN or Inf to zero
    x = np.where(np.isnan(x), 0.0, x)
    x = np.where(np.isinf(x), 0.0, x)
    
    return x

def preprocess_rec_traj(x):
    """
    :description: Run preprocessing on input array assuming it is full matrix of REC::Traj bank.
    
    :param: x
    
    :return: new_x
    """
    #NOTE: Assume indices go 0-10 : pindex, index, detector, layer, x, y, z, cx, cy, cz, path
    
    pindex_idx, index_idx, detector_idx, layer_idx, x_idx, y_idx, z_idx, cx_idx, cy_idx, cz_idx, path_idx = [i for i in range(np.shape(x)[-1])]
    
    # Preprocess row info
#     x[:,x_idx]    -= 0.0
    x[:,x_idx]    /= 1500.0
#     x[:,y_idx]    -= 0.0
    x[:,y_idx]    /= 1500.0
    x[:,z_idx]    -= 1500.0/2
    x[:,z_idx]    /= 1500.0
#     x[:,cx_idx]   -= 0.0
#     x[:,cx_idx]   /= 1.0
#     x[:,cy_idx]   -= 0.0
#     x[:,cy_idx]   /= 1.0
    x[:,cz_idx]   -= 1.0
    x[:,cz_idx]   /= 2.0
    x[:,path_idx] -= 1500.0/2
    x[:,path_idx] /= 1500.0
    
    # Select rows to take #NOTE: Do this after preprocessing so indices make sense
    indices = [x_idx, y_idx, z_idx, cx_idx, cy_idx, cz_idx, path_idx]
    x = np.take(x,indices,axis=1)
    
    # Reassign all data values that are NaN or Inf to zero
    x = np.where(np.isnan(x), 0.0, x)
    x = np.where(np.isinf(x), 0.0, x)
    
    return x

def get_links_rec_traj(x):
    
    pindex = 0
    link_indices = []
    prev_el = -10
    for idx, el in enumerate(x[:,pindex]):
        if el != prev_el:
            link_indices.append([idx])
        else:
            link_indices[-1].append(idx)
        prev_el = el
        
    return link_indices

def get_edge_index_rec_traj(x):
    
    link_indices = get_links_rec_traj(x)
    
    edge_index = torch.tensor([[i,j] for el_ in link_indices for i in el_ for j in el_],dtype=torch.long)
        
    return edge_index

x___ = torch.tensor([[i//3] for i in range(15)],dtype=torch.long)  


In [10]:
1+1

2

In [12]:
import random
random.gauss(-2.598, 2*3.394)    

1.0004980358138686

In [14]:
print('/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_0.hipo')
print('/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_'+str(0)+'.hipo')

/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_0.hipo
/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_0.hipo


In [19]:
#for i in range(15, 15):  
print(i)
filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_'+str(i)+'.hipo'
step = 40000
for batch in hp.iterate([filee],banks=["MC::Lund", "REC::Particle", "REC::Traj", "REC::Kinematics"],step=step): 
    print(batch.keys()) 
    #batch.keys()
    dic = batch.keys() 
    break   

7085


ValueError: HipoFileIterator: length of filenames must be > 0

In [17]:
1+1

2

In [ ]:
#mkaing half and half data sets, half is lambda combinning two whole sets, third is 10K of it without the lambda force
#modified now to be quaoter datasets. 
from torch_geometric.data import Dataset, download_url, InMemoryDataset

 #third is list of graphss   
third = []
kkkk = 0
#for j in range(1):    
 #   number = 5
    #if j==2:
      #  number = 2
j=0  
number = 5
#for j in range(3):
 #   if j==3:
 #       number = 2
for ii in range(number): 
    if j==0:
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_'+str(ii)+'.hipo'
    if j==1:
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3316_'+str(ii)+'.hipo'
    if j==2:
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3317_'+str(ii)+'.hipo'
    if j==3 :
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3318_'+str(ii)+'.hipo'
    if j==4:
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3327_'+str(ii)+'.hipo'

    step = 40000
    for batch in hp.iterate([filee],banks=["MC::Lund", "REC::Particle", "REC::Traj", "REC::Kinematics"],step=step): 
       # print(batch.keys()) 
        #batch.keys()
        dic = batch.keys() 
        break   

    #for battch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_0.hipo'],banks=["REC::Kinematics"],step=step):
        #print(battch.keys()) 
        #batch.keys()
        #dic = battch.keys() 
       # break  
    #
    all_keys            = list(batch.keys())
    rec_particle_name   = 'REC::Particle'
    rec_particle_keys   = get_bank_keys(rec_particle_name,all_keys)
    rec_traj_name       = 'REC::Traj'
    rec_traj_keys       = get_bank_keys(rec_traj_name,all_keys)
    rec_kinematics_name = 'REC::Kinematics' 
    rec_kinematics_keys = get_bank_keys(rec_kinematics_name,all_keys)
    mc_lund_name        = 'MC::Lund'
    mc_lund_keys        = get_bank_keys(mc_lund_name,all_keys) 

    # rec_kinematics_keys = ['REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 
    #                       'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass']
    #fulll = [] 
    tst = []
    y=[]


    #j is proton, kk is pion


    kinn = []  
    idkkk =[]
    for i in range(step):
        # Get REC::Particle bank
        rec_particle_event_table = get_event_table(rec_particle_keys,i,batch,dtype=float)

        # Get REC::Traj bank
        rec_traj_event_table = get_event_table(rec_traj_keys,i,batch,dtype=float)

        # Get REC::Kinematics bank
        rec_kinematics_event_table = get_event_table(rec_kinematics_keys,i,batch,dtype=float)

        # Get MC::Lund bank and MC->REC matching indices
        mc_lund_event_table = get_event_table(mc_lund_keys,i,batch,dtype=float)
        match_indices  = get_match_indices(rec_particle_event_table,mc_lund_event_table)

        decay = [3122,[2212,-211]] 
        has_decay, rec_indices = check_has_decay(rec_particle_event_table,mc_lund_event_table,match_indices,decay,
                                       rec_particle_pid_idx=0,mc_lund_pid_idx=3,mc_lund_parent_idx=4,mc_lund_daughter_idx=5)

        #print(rec_indices)
        #getting the index of where the electron proton and pion r 
        savee = batch['REC::Particle_pid'][i]
        j = savee.index(2212); kk = savee.index(-211); elc = savee.index(11) #elc is electron, used for checking that it matches up, j is proton, kk is pionn 
        #ckeck
        #making sure that it is not bad (for the match) #if is bad then lots more stuff to do 
        if match_indices[kk][1]!=-1: 
            #if (np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)<1):
            if kk in rec_indices and has_decay == True:    
                track_hit = []
                indexp = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == j]
                indexpi = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == kk]
                #get all the data we need here. 

                totalpos =[ [batch['REC::Traj_x'][i][ii] for ii in indexpi], [ batch['REC::Traj_y'][i][ii] for ii in indexpi], [ batch['REC::Traj_z'][i][ii] for ii in indexpi],
                          [batch['REC::Traj_cx'][i][ii] for ii in indexpi], [ batch['REC::Traj_cy'][i][ii] for ii in indexpi], [ batch['REC::Traj_cz'][i][ii] for ii in indexpi]]


                s = 0
                ss =0 
                for dd in range(len(indexpi)):
                    track_hit += [[batch['REC::Traj_x'][i][indexpi[dd]], batch['REC::Traj_y'][i][indexpi[dd]], batch['REC::Traj_z'][i][indexpi[dd]],
                                 batch['REC::Traj_cx'][i][indexpi[dd]], batch['REC::Traj_cy'][i][indexpi[dd]], batch['REC::Traj_cz'][i][indexpi[dd]],
                                   batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk]]]
                                 # batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk],
                                   #random.gauss(batch['REC::Particle_px'][i][kk], 1), random.gauss(batch['REC::Particle_py'][i][kk], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][kk], 1),
                                  # random.gauss(batch['REC::Particle_vx'][i][kk], 3.394), random.gauss(batch['REC::Particle_vy'][i][kk], 3.394), 
                                  # random.gauss(batch['REC::Particle_vz'][i][kk], 3.394)]]
                    s +=1
                #track_hit+=[[batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                 #           batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk]]]
                ss =0 
                for dd in range(len(indexp)): 
                    track_hit += [[batch['REC::Traj_x'][i][indexp[dd]], batch['REC::Traj_y'][i][indexp[dd]], batch['REC::Traj_z'][i][indexp[dd]],
                                   batch['REC::Traj_cx'][i][indexp[dd]], batch['REC::Traj_cy'][i][indexp[dd]], batch['REC::Traj_cz'][i][indexp[dd]],
                                  batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j]]]
                                  #batch['REC::Particle_vx'][i][j], batch['REC::Particle_vy'][i][j], batch['REC::Particle_vz'][i][j],
                                   #random.gauss(batch['REC::Particle_px'][i][j], 1), random.gauss(batch['REC::Particle_py'][i][j], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][j], 1),
                                  # random.gauss(batch['REC::Particle_vx'][i][j], 3.394), random.gauss(batch['REC::Particle_vy'][i][j], 3.394), 
                                  # random.gauss(batch['REC::Particle_vz'][i][j], 3.394)]]

                    ss+=1  


                y = [batch['MC::Lund_vz'][i][match_indices[kk][1]] ] 
                #y = [np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)]  

                #track_hit += [[np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)]]

                start = []
                end = []
                for gg in range(s-1):
                    start.append(gg)
                    end.append(gg+1)
                for ggg in range(ss-1): 
                    start.append(s+ggg) 
                    end.append(s+ggg+1)
                #fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
                fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
               # fourth.kinematics = [battch['REC::Kinematics_Q2'][i][0], battch['REC::Kinematics_nu'][i][0], battch['REC::Kinematics_W'][i][0], battch['REC::Kinematics_x'][i][0],  
                #                     battch['REC::Kinematics_y'][i][0], battch['REC::Kinematics_z'][i][0], battch['REC::Kinematics_xF'][i][0], battch['REC::Kinematics_mass'][i][0]]
                fourth.rec  = [batch['REC::Particle_vz'][i][kk]] 
                #fourth.mom = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                fourth.elc = [batch['REC::Particle_vz'][i][0] ] 
                fourth.momentum = [batch['MC::Lund_pz'][i][match_indices[kk][1]]]
                #fourth.rec = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                #idkkk+=[[batch['REC::Particle_vz'][i][kk], batch['REC::Particle_vz'][i][j]]]
                #invarient masss#######################################################################################################
                fourth.imass = [batch['REC::Kinematics_mass'][i][0]] 
                if kk in rec_indices and has_decay == True:
                    fourth.LamT = [1]
                else :
                    fourth.LamT = [0]
                third+= [fourth]
    kkkk+=1
    print(kkkk) 
ii=2 
if j==0:
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_'+str(ii)+'.hipo'
if j==1:
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3316_'+str(ii)+'.hipo'
if j==2:
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3317_'+str(ii)+'.hipo'
if j==3 :
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3318_'+str(ii)+'.hipo'
if j==4:
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3327_'+str(ii)+'.hipo'

step = 40000   
for batch in hp.iterate([filee],banks=["MC::Lund", "REC::Particle", "REC::Traj", "REC::Kinematics"],step=step): 
   # print(batch.keys()) 
    #batch.keys()
    dic = batch.keys() 
    break   

#for battch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_0.hipo'],banks=["REC::Kinematics"],step=step):
    #print(battch.keys()) 
    #batch.keys()
    #dic = battch.keys() 
   # break  
#
all_keys            = list(batch.keys())
rec_particle_name   = 'REC::Particle'
rec_particle_keys   = get_bank_keys(rec_particle_name,all_keys)
rec_traj_name       = 'REC::Traj'
rec_traj_keys       = get_bank_keys(rec_traj_name,all_keys)
rec_kinematics_name = 'REC::Kinematics' 
rec_kinematics_keys = get_bank_keys(rec_kinematics_name,all_keys)
mc_lund_name        = 'MC::Lund'
mc_lund_keys        = get_bank_keys(mc_lund_name,all_keys) 

# rec_kinematics_keys = ['REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 
#                       'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass']
#fulll = [] 
tst = []
y=[]


#j is proton, kk is pion


kinn = []  
idkkk =[]
for i in range(step):
    # Get REC::Particle bank
    rec_particle_event_table = get_event_table(rec_particle_keys,i,batch,dtype=float)

    # Get REC::Traj bank
    rec_traj_event_table = get_event_table(rec_traj_keys,i,batch,dtype=float)

    # Get REC::Kinematics bank
    rec_kinematics_event_table = get_event_table(rec_kinematics_keys,i,batch,dtype=float)

    # Get MC::Lund bank and MC->REC matching indices
    mc_lund_event_table = get_event_table(mc_lund_keys,i,batch,dtype=float)
    match_indices  = get_match_indices(rec_particle_event_table,mc_lund_event_table)

    decay = [3122,[2212,-211]] 
    has_decay, rec_indices = check_has_decay(rec_particle_event_table,mc_lund_event_table,match_indices,decay,
                                   rec_particle_pid_idx=0,mc_lund_pid_idx=3,mc_lund_parent_idx=4,mc_lund_daughter_idx=5)

    #print(rec_indices)
    #getting the index of where the electron proton and pion r 
    savee = batch['REC::Particle_pid'][i]
    j = savee.index(2212); kk = savee.index(-211); elc = savee.index(11) #elc is electron, used for checking that it matches up, j is proton, kk is pionn 
    #ckeck
    #making sure that it is not bad (for the match) #if is bad then lots more stuff to do 
    if match_indices[kk][1]!=-1: 
        #if (np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)<1):
        #if kk in rec_indices and has_decay == True:     
        track_hit = []
        indexp = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == j]
        indexpi = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == kk]
        #get all the data we need here. 

        totalpos =[ [batch['REC::Traj_x'][i][ii] for ii in indexpi], [ batch['REC::Traj_y'][i][ii] for ii in indexpi], [ batch['REC::Traj_z'][i][ii] for ii in indexpi],
                  [batch['REC::Traj_cx'][i][ii] for ii in indexpi], [ batch['REC::Traj_cy'][i][ii] for ii in indexpi], [ batch['REC::Traj_cz'][i][ii] for ii in indexpi]]


        s = 0
        ss =0 
        for dd in range(len(indexpi)):
            track_hit += [[batch['REC::Traj_x'][i][indexpi[dd]], batch['REC::Traj_y'][i][indexpi[dd]], batch['REC::Traj_z'][i][indexpi[dd]],
                         batch['REC::Traj_cx'][i][indexpi[dd]], batch['REC::Traj_cy'][i][indexpi[dd]], batch['REC::Traj_cz'][i][indexpi[dd]],
                           batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk]]]
                         # batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk],
                           #random.gauss(batch['REC::Particle_px'][i][kk], 1), random.gauss(batch['REC::Particle_py'][i][kk], 1), 
                           #random.gauss(batch['REC::Particle_pz'][i][kk], 1),
                          # random.gauss(batch['REC::Particle_vx'][i][kk], 3.394), random.gauss(batch['REC::Particle_vy'][i][kk], 3.394), 
                          # random.gauss(batch['REC::Particle_vz'][i][kk], 3.394)]]
            s +=1
        #track_hit+=[[batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
         #           batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk]]]
        ss =0 
        for dd in range(len(indexp)): 
            track_hit += [[batch['REC::Traj_x'][i][indexp[dd]], batch['REC::Traj_y'][i][indexp[dd]], batch['REC::Traj_z'][i][indexp[dd]],
                           batch['REC::Traj_cx'][i][indexp[dd]], batch['REC::Traj_cy'][i][indexp[dd]], batch['REC::Traj_cz'][i][indexp[dd]],
                          batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j]]]
                          #batch['REC::Particle_vx'][i][j], batch['REC::Particle_vy'][i][j], batch['REC::Particle_vz'][i][j],
                           #random.gauss(batch['REC::Particle_px'][i][j], 1), random.gauss(batch['REC::Particle_py'][i][j], 1), 
                           #random.gauss(batch['REC::Particle_pz'][i][j], 1),
                           #random.gauss(batch['REC::Particle_vx'][i][j], 3.394), random.gauss(batch['REC::Particle_vy'][i][j], 3.394), 
                          # random.gauss(batch['REC::Particle_vz'][i][j], 3.394)]]

            ss+=1  


        y = [batch['MC::Lund_vz'][i][match_indices[kk][1]] ] 
        #y = [np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)]  

        #track_hit += [[np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)]]

        start = []
        end = []
        for gg in range(s-1):
            start.append(gg)
            end.append(gg+1)
        for ggg in range(ss-1): 
            start.append(s+ggg) 
            end.append(s+ggg+1)
        #fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
        fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
       # fourth.kinematics = [battch['REC::Kinematics_Q2'][i][0], battch['REC::Kinematics_nu'][i][0], battch['REC::Kinematics_W'][i][0], battch['REC::Kinematics_x'][i][0],  
        #                     battch['REC::Kinematics_y'][i][0], battch['REC::Kinematics_z'][i][0], battch['REC::Kinematics_xF'][i][0], battch['REC::Kinematics_mass'][i][0]]
        fourth.rec  = [batch['REC::Particle_vz'][i][kk]] 
        #fourth.mom = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
        fourth.elc = [batch['REC::Particle_vz'][i][0] ] 
        #fourth.rec = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
        #idkkk+=[[batch['REC::Particle_vz'][i][kk], batch['REC::Particle_vz'][i][j]]]
        #invarient masss#######################################################################################################
        fourth.imass = [batch['REC::Kinematics_mass'][i][0]] 
        if kk in rec_indices and has_decay == True:
            fourth.LamT = [1]
        else :
            fourth.LamT = [0]
        third+= [fourth]
kkkk+=1
print(kkkk) 

data_list = third  
root = '/work/clas12/users/mencke/dikkk'           
class MyDataset(InMemoryDataset):
    def __init__(self, root, data_list, transform=None): 
        self.data_list = data_list
        super().__init__(root, transform)  
        self.load(self.processed_paths[0])

    @property
    def processed_file_names(self): 
        return 'data.pt' 
 
    def process(self):
        self.save(self.data_list, self.processed_paths[0])      

MyDataset(root, third)  


In [16]:
#mkaing half and half data sets, half is lambda combinning two whole sets, third is 10K of it without the lambda force
#modified now to be quaoter datasets. 
from torch_geometric.data import Dataset, download_url, InMemoryDataset

 #third is list of graphss   
third = []
kkkk = 0
#for j in range(1):    
 #   number = 5
    #if j==2:
      #  number = 2
j=0  
number = 2

for ii in range(number): 
    if j==0:
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_'+str(ii)+'.hipo'
    if j==1:
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3316_'+str(ii)+'.hipo'
    if j==2:
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3317_'+str(ii)+'.hipo'
    if j==3 :
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3318_'+str(ii)+'.hipo'
    if j==4:
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3327_'+str(ii)+'.hipo'

    step = 40000
    for batch in hp.iterate([filee],banks=["MC::Lund", "REC::Particle", "REC::Traj", "REC::Kinematics"],step=step): 
       # print(batch.keys()) 
        #batch.keys()
        dic = batch.keys() 
        break   

    #for battch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_0.hipo'],banks=["REC::Kinematics"],step=step):
        #print(battch.keys()) 
        #batch.keys()
        #dic = battch.keys() 
       # break  
    #
    all_keys            = list(batch.keys())
    rec_particle_name   = 'REC::Particle'
    rec_particle_keys   = get_bank_keys(rec_particle_name,all_keys)
    rec_traj_name       = 'REC::Traj'
    rec_traj_keys       = get_bank_keys(rec_traj_name,all_keys)
    rec_kinematics_name = 'REC::Kinematics' 
    rec_kinematics_keys = get_bank_keys(rec_kinematics_name,all_keys)
    mc_lund_name        = 'MC::Lund'
    mc_lund_keys        = get_bank_keys(mc_lund_name,all_keys) 

    # rec_kinematics_keys = ['REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 
    #                       'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass']
    #fulll = [] 
    tst = []
    y=[]


    #j is proton, kk is pion


    kinn = []  
    idkkk =[]
    for i in range(step):
        # Get REC::Particle bank
        rec_particle_event_table = get_event_table(rec_particle_keys,i,batch,dtype=float)

        # Get REC::Traj bank
        rec_traj_event_table = get_event_table(rec_traj_keys,i,batch,dtype=float)

        # Get REC::Kinematics bank
        rec_kinematics_event_table = get_event_table(rec_kinematics_keys,i,batch,dtype=float)

        # Get MC::Lund bank and MC->REC matching indices
        mc_lund_event_table = get_event_table(mc_lund_keys,i,batch,dtype=float)
        match_indices  = get_match_indices(rec_particle_event_table,mc_lund_event_table)

        decay = [3122,[2212,-211]] 
        has_decay, rec_indices = check_has_decay(rec_particle_event_table,mc_lund_event_table,match_indices,decay,
                                       rec_particle_pid_idx=0,mc_lund_pid_idx=3,mc_lund_parent_idx=4,mc_lund_daughter_idx=5)

        #print(rec_indices)
        #getting the index of where the electron proton and pion r 
        savee = batch['REC::Particle_pid'][i]
        j = savee.index(2212); kk = savee.index(-211); elc = savee.index(11) #elc is electron, used for checking that it matches up, j is proton, kk is pionn 
        #ckeck
        #making sure that it is not bad (for the match) #if is bad then lots more stuff to do 
        if match_indices[kk][1]!=-1: 
            #if (np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)<1):
            if kk in rec_indices and has_decay == True:    
                track_hit = []
                indexp = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == j]
                indexpi = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == kk]
                #get all the data we need here. 

                totalpos =[ [batch['REC::Traj_x'][i][ii] for ii in indexpi], [ batch['REC::Traj_y'][i][ii] for ii in indexpi], [ batch['REC::Traj_z'][i][ii] for ii in indexpi],
                          [batch['REC::Traj_cx'][i][ii] for ii in indexpi], [ batch['REC::Traj_cy'][i][ii] for ii in indexpi], [ batch['REC::Traj_cz'][i][ii] for ii in indexpi]]


                s = 0
                ss =0 
                for dd in range(len(indexpi)):
                    track_hit += [[batch['REC::Traj_x'][i][indexpi[dd]], batch['REC::Traj_y'][i][indexpi[dd]], batch['REC::Traj_z'][i][indexpi[dd]],
                                 batch['REC::Traj_cx'][i][indexpi[dd]], batch['REC::Traj_cy'][i][indexpi[dd]], batch['REC::Traj_cz'][i][indexpi[dd]],
                                   batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk]]]
                                 # batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk],
                                   #random.gauss(batch['REC::Particle_px'][i][kk], 1), random.gauss(batch['REC::Particle_py'][i][kk], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][kk], 1),
                                  # random.gauss(batch['REC::Particle_vx'][i][kk], 3.394), random.gauss(batch['REC::Particle_vy'][i][kk], 3.394), 
                                  # random.gauss(batch['REC::Particle_vz'][i][kk], 3.394)]]
                    s +=1
                #track_hit+=[[batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                 #           batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk]]]
                ss =0 
                for dd in range(len(indexp)): 
                    track_hit += [[batch['REC::Traj_x'][i][indexp[dd]], batch['REC::Traj_y'][i][indexp[dd]], batch['REC::Traj_z'][i][indexp[dd]],
                                   batch['REC::Traj_cx'][i][indexp[dd]], batch['REC::Traj_cy'][i][indexp[dd]], batch['REC::Traj_cz'][i][indexp[dd]],
                                  batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j]]]
                                  #batch['REC::Particle_vx'][i][j], batch['REC::Particle_vy'][i][j], batch['REC::Particle_vz'][i][j],
                                   #random.gauss(batch['REC::Particle_px'][i][j], 1), random.gauss(batch['REC::Particle_py'][i][j], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][j], 1),
                                  # random.gauss(batch['REC::Particle_vx'][i][j], 3.394), random.gauss(batch['REC::Particle_vy'][i][j], 3.394), 
                                  # random.gauss(batch['REC::Particle_vz'][i][j], 3.394)]]

                    ss+=1  


                y = [batch['MC::Lund_vz'][i][match_indices[kk][1]] ] 
                #y = [np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)]  

                #track_hit += [[np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)]]

                start = []
                end = []
                for gg in range(s-1):
                    start.append(gg)
                    end.append(gg+1)
                for ggg in range(ss-1): 
                    start.append(s+ggg) 
                    end.append(s+ggg+1)
                #fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
                fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
               # fourth.kinematics = [battch['REC::Kinematics_Q2'][i][0], battch['REC::Kinematics_nu'][i][0], battch['REC::Kinematics_W'][i][0], battch['REC::Kinematics_x'][i][0],  
                #                     battch['REC::Kinematics_y'][i][0], battch['REC::Kinematics_z'][i][0], battch['REC::Kinematics_xF'][i][0], battch['REC::Kinematics_mass'][i][0]]
                fourth.rec  = [batch['REC::Particle_vz'][i][kk]] 
                #fourth.mom = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                fourth.elc = [batch['REC::Particle_vz'][i][0] ] 
                #fourth.rec = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                #idkkk+=[[batch['REC::Particle_vz'][i][kk], batch['REC::Particle_vz'][i][j]]]
                #invarient masss#######################################################################################################
                fourth.imass = [batch['REC::Kinematics_mass'][i][0]] 
                if kk in rec_indices and has_decay == True:
                    fourth.LamT = [1]
                else :
                    fourth.LamT = [0]
                third+= [fourth]
    kkkk+=1
    print(kkkk) 
ii=2 
if j==0:
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_'+str(ii)+'.hipo'
if j==1:
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3316_'+str(ii)+'.hipo'
if j==2:
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3317_'+str(ii)+'.hipo'
if j==3 :
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3318_'+str(ii)+'.hipo'
if j==4:
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3327_'+str(ii)+'.hipo'

step = 12000   
for batch in hp.iterate([filee],banks=["MC::Lund", "REC::Particle", "REC::Traj", "REC::Kinematics"],step=step): 
   # print(batch.keys()) 
    #batch.keys()
    dic = batch.keys() 
    break   

#for battch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_0.hipo'],banks=["REC::Kinematics"],step=step):
    #print(battch.keys()) 
    #batch.keys()
    #dic = battch.keys() 
   # break  
#
all_keys            = list(batch.keys())
rec_particle_name   = 'REC::Particle'
rec_particle_keys   = get_bank_keys(rec_particle_name,all_keys)
rec_traj_name       = 'REC::Traj'
rec_traj_keys       = get_bank_keys(rec_traj_name,all_keys)
rec_kinematics_name = 'REC::Kinematics' 
rec_kinematics_keys = get_bank_keys(rec_kinematics_name,all_keys)
mc_lund_name        = 'MC::Lund'
mc_lund_keys        = get_bank_keys(mc_lund_name,all_keys) 

# rec_kinematics_keys = ['REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 
#                       'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass']
#fulll = [] 
tst = []
y=[]


#j is proton, kk is pion


kinn = []  
idkkk =[]
for i in range(step):
    # Get REC::Particle bank
    rec_particle_event_table = get_event_table(rec_particle_keys,i,batch,dtype=float)

    # Get REC::Traj bank
    rec_traj_event_table = get_event_table(rec_traj_keys,i,batch,dtype=float)

    # Get REC::Kinematics bank
    rec_kinematics_event_table = get_event_table(rec_kinematics_keys,i,batch,dtype=float)

    # Get MC::Lund bank and MC->REC matching indices
    mc_lund_event_table = get_event_table(mc_lund_keys,i,batch,dtype=float)
    match_indices  = get_match_indices(rec_particle_event_table,mc_lund_event_table)

    decay = [3122,[2212,-211]] 
    has_decay, rec_indices = check_has_decay(rec_particle_event_table,mc_lund_event_table,match_indices,decay,
                                   rec_particle_pid_idx=0,mc_lund_pid_idx=3,mc_lund_parent_idx=4,mc_lund_daughter_idx=5)

    #print(rec_indices)
    #getting the index of where the electron proton and pion r 
    savee = batch['REC::Particle_pid'][i]
    j = savee.index(2212); kk = savee.index(-211); elc = savee.index(11) #elc is electron, used for checking that it matches up, j is proton, kk is pionn 
    #ckeck
    #making sure that it is not bad (for the match) #if is bad then lots more stuff to do 
    if match_indices[kk][1]!=-1: 
        #if (np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)<1):
        #if kk in rec_indices and has_decay == True:     
        track_hit = []
        indexp = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == j]
        indexpi = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == kk]
        #get all the data we need here. 

        totalpos =[ [batch['REC::Traj_x'][i][ii] for ii in indexpi], [ batch['REC::Traj_y'][i][ii] for ii in indexpi], [ batch['REC::Traj_z'][i][ii] for ii in indexpi],
                  [batch['REC::Traj_cx'][i][ii] for ii in indexpi], [ batch['REC::Traj_cy'][i][ii] for ii in indexpi], [ batch['REC::Traj_cz'][i][ii] for ii in indexpi]]


        s = 0
        ss =0 
        for dd in range(len(indexpi)):
            track_hit += [[batch['REC::Traj_x'][i][indexpi[dd]], batch['REC::Traj_y'][i][indexpi[dd]], batch['REC::Traj_z'][i][indexpi[dd]],
                         batch['REC::Traj_cx'][i][indexpi[dd]], batch['REC::Traj_cy'][i][indexpi[dd]], batch['REC::Traj_cz'][i][indexpi[dd]],
                           batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk]]]
                         # batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk],
                           #random.gauss(batch['REC::Particle_px'][i][kk], 1), random.gauss(batch['REC::Particle_py'][i][kk], 1), 
                           #random.gauss(batch['REC::Particle_pz'][i][kk], 1),
                          # random.gauss(batch['REC::Particle_vx'][i][kk], 3.394), random.gauss(batch['REC::Particle_vy'][i][kk], 3.394), 
                          # random.gauss(batch['REC::Particle_vz'][i][kk], 3.394)]]
            s +=1
        #track_hit+=[[batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
         #           batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk]]]
        ss =0 
        for dd in range(len(indexp)): 
            track_hit += [[batch['REC::Traj_x'][i][indexp[dd]], batch['REC::Traj_y'][i][indexp[dd]], batch['REC::Traj_z'][i][indexp[dd]],
                           batch['REC::Traj_cx'][i][indexp[dd]], batch['REC::Traj_cy'][i][indexp[dd]], batch['REC::Traj_cz'][i][indexp[dd]],
                          batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j]]]
                          #batch['REC::Particle_vx'][i][j], batch['REC::Particle_vy'][i][j], batch['REC::Particle_vz'][i][j],
                           #random.gauss(batch['REC::Particle_px'][i][j], 1), random.gauss(batch['REC::Particle_py'][i][j], 1), 
                           #random.gauss(batch['REC::Particle_pz'][i][j], 1),
                           #random.gauss(batch['REC::Particle_vx'][i][j], 3.394), random.gauss(batch['REC::Particle_vy'][i][j], 3.394), 
                          # random.gauss(batch['REC::Particle_vz'][i][j], 3.394)]]

            ss+=1  


        y = [batch['MC::Lund_vz'][i][match_indices[kk][1]] ] 
        #y = [np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)]  

        #track_hit += [[np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)]]

        start = []
        end = []
        for gg in range(s-1):
            start.append(gg)
            end.append(gg+1)
        for ggg in range(ss-1): 
            start.append(s+ggg) 
            end.append(s+ggg+1)
        #fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
        fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
       # fourth.kinematics = [battch['REC::Kinematics_Q2'][i][0], battch['REC::Kinematics_nu'][i][0], battch['REC::Kinematics_W'][i][0], battch['REC::Kinematics_x'][i][0],  
        #                     battch['REC::Kinematics_y'][i][0], battch['REC::Kinematics_z'][i][0], battch['REC::Kinematics_xF'][i][0], battch['REC::Kinematics_mass'][i][0]]
        fourth.rec  = [batch['REC::Particle_vz'][i][kk]] 
        #fourth.mom = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
        fourth.elc = [batch['REC::Particle_vz'][i][0] ] 
        #fourth.rec = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
        #idkkk+=[[batch['REC::Particle_vz'][i][kk], batch['REC::Particle_vz'][i][j]]]
        #invarient masss#######################################################################################################
        fourth.imass = [batch['REC::Kinematics_mass'][i][0]] 
        if kk in rec_indices and has_decay == True:
            fourth.LamT = [1]
        else :
            fourth.LamT = [0]
        third+= [fourth]
kkkk+=1
print(kkkk) 

data_list = third  
root = '/work/clas12/users/mencke/dikkk'           
class MyDataset(InMemoryDataset):
    def __init__(self, root, data_list, transform=None): 
        self.data_list = data_list
        super().__init__(root, transform)  
        self.load(self.processed_paths[0])

    @property
    def processed_file_names(self): 
        return 'data.pt' 
 
    def process(self):
        self.save(self.data_list, self.processed_paths[0])      

MyDataset(root, third)       

KeyboardInterrupt: 

In [ ]:
root = '/work/clas12/users/mencke/Hnorec_Last', j=0 
root = '/work/clas12/users/mencke/Hnorec_first', j=1
root = '/work/clas12/users/mencke/Hnorec_second', j=2
oot = '/work/clas12/users/mencke/Hnorec_third' , j=3
/work/clas12/users/mencke/Hnorec_fourth' , j=4

In [3]:
#mkaing half and half data sets, half is lambda combinning two whole sets, third is 10K of it without the lambda force
#modified now to be quaoter datasets. 
from torch_geometric.data import Dataset, download_url, InMemoryDataset

 #third is list of graphss   
third = []
kkkk = 0
#for j in range(1):    
 #   number = 5
    #if j==2:
      #  number = 2
j=4  
number = 1

for ii in range(number): 
    if j==0:
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_'+str(ii)+'.hipo'
    if j==1:
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3316_'+str(ii)+'.hipo'
    if j==2:
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3317_'+str(ii)+'.hipo'
    if j==3 :
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3318_'+str(ii)+'.hipo'
    if j==4:
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3327_'+str(ii)+'.hipo'

    step = 40000
    for batch in hp.iterate([filee],banks=["MC::Lund", "REC::Particle", "REC::Traj", "REC::Kinematics"],step=step): 
       # print(batch.keys()) 
        #batch.keys()
        dic = batch.keys() 
        break   

    #for battch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_0.hipo'],banks=["REC::Kinematics"],step=step):
        #print(battch.keys()) 
        #batch.keys()
        #dic = battch.keys() 
       # break  
    #
    all_keys            = list(batch.keys())
    rec_particle_name   = 'REC::Particle'
    rec_particle_keys   = get_bank_keys(rec_particle_name,all_keys)
    rec_traj_name       = 'REC::Traj'
    rec_traj_keys       = get_bank_keys(rec_traj_name,all_keys)
    rec_kinematics_name = 'REC::Kinematics' 
    rec_kinematics_keys = get_bank_keys(rec_kinematics_name,all_keys)
    mc_lund_name        = 'MC::Lund'
    mc_lund_keys        = get_bank_keys(mc_lund_name,all_keys) 

    # rec_kinematics_keys = ['REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 
    #                       'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass']
    #fulll = [] 
    tst = []
    y=[]


    #j is proton, kk is pion


    kinn = []  
    idkkk =[]
    for i in range(step):
        # Get REC::Particle bank
        rec_particle_event_table = get_event_table(rec_particle_keys,i,batch,dtype=float)

        # Get REC::Traj bank
        rec_traj_event_table = get_event_table(rec_traj_keys,i,batch,dtype=float)

        # Get REC::Kinematics bank
        rec_kinematics_event_table = get_event_table(rec_kinematics_keys,i,batch,dtype=float)

        # Get MC::Lund bank and MC->REC matching indices
        mc_lund_event_table = get_event_table(mc_lund_keys,i,batch,dtype=float)
        match_indices  = get_match_indices(rec_particle_event_table,mc_lund_event_table)

        decay = [3122,[2212,-211]] 
        has_decay, rec_indices = check_has_decay(rec_particle_event_table,mc_lund_event_table,match_indices,decay,
                                       rec_particle_pid_idx=0,mc_lund_pid_idx=3,mc_lund_parent_idx=4,mc_lund_daughter_idx=5)

        #print(rec_indices)
        #getting the index of where the electron proton and pion r 
        savee = batch['REC::Particle_pid'][i]
        j = savee.index(2212); kk = savee.index(-211); elc = savee.index(11) #elc is electron, used for checking that it matches up, j is proton, kk is pionn 
        #ckeck
        #making sure that it is not bad (for the match) #if is bad then lots more stuff to do 
        if match_indices[kk][1]!=-1: 
            #if (np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)<1):
            if kk in rec_indices and has_decay == True:    
                track_hit = []
                indexp = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == j]
                indexpi = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == kk]
                #get all the data we need here. 

                totalpos =[ [batch['REC::Traj_x'][i][ii] for ii in indexpi], [ batch['REC::Traj_y'][i][ii] for ii in indexpi], [ batch['REC::Traj_z'][i][ii] for ii in indexpi],
                          [batch['REC::Traj_cx'][i][ii] for ii in indexpi], [ batch['REC::Traj_cy'][i][ii] for ii in indexpi], [ batch['REC::Traj_cz'][i][ii] for ii in indexpi]]


                s = 0
                ss =0 
                for dd in range(len(indexpi)):
                    track_hit += [[batch['REC::Traj_x'][i][indexpi[dd]], batch['REC::Traj_y'][i][indexpi[dd]], batch['REC::Traj_z'][i][indexpi[dd]],
                                 batch['REC::Traj_cx'][i][indexpi[dd]], batch['REC::Traj_cy'][i][indexpi[dd]], batch['REC::Traj_cz'][i][indexpi[dd]],
                                   batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                                 # batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk],
                                   #random.gauss(batch['REC::Particle_px'][i][kk], 1), random.gauss(batch['REC::Particle_py'][i][kk], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][kk], 1),
                                   random.gauss(batch['REC::Particle_vx'][i][kk], 3.394), random.gauss(batch['REC::Particle_vy'][i][kk], 3.394), 
                                   random.gauss(batch['REC::Particle_vz'][i][kk], 3.394)]]
                    s +=1
                #track_hit+=[[batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                 #           batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk]]]
                ss =0 
                for dd in range(len(indexp)): 
                    track_hit += [[batch['REC::Traj_x'][i][indexp[dd]], batch['REC::Traj_y'][i][indexp[dd]], batch['REC::Traj_z'][i][indexp[dd]],
                                   batch['REC::Traj_cx'][i][indexp[dd]], batch['REC::Traj_cy'][i][indexp[dd]], batch['REC::Traj_cz'][i][indexp[dd]],
                                  batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j],
                                  #batch['REC::Particle_vx'][i][j], batch['REC::Particle_vy'][i][j], batch['REC::Particle_vz'][i][j],
                                   #random.gauss(batch['REC::Particle_px'][i][j], 1), random.gauss(batch['REC::Particle_py'][i][j], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][j], 1),
                                   random.gauss(batch['REC::Particle_vx'][i][j], 3.394), random.gauss(batch['REC::Particle_vy'][i][j], 3.394), 
                                   random.gauss(batch['REC::Particle_vz'][i][j], 3.394)]]

                    ss+=1  


                y = [batch['MC::Lund_vz'][i][match_indices[kk][1]] ] 
                #y = [np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)]  

                #track_hit += [[np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)]]

                start = []
                end = []
                for gg in range(s-1):
                    start.append(gg)
                    end.append(gg+1)
                for ggg in range(ss-1): 
                    start.append(s+ggg) 
                    end.append(s+ggg+1)
                #fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
                fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
               # fourth.kinematics = [battch['REC::Kinematics_Q2'][i][0], battch['REC::Kinematics_nu'][i][0], battch['REC::Kinematics_W'][i][0], battch['REC::Kinematics_x'][i][0],  
                #                     battch['REC::Kinematics_y'][i][0], battch['REC::Kinematics_z'][i][0], battch['REC::Kinematics_xF'][i][0], battch['REC::Kinematics_mass'][i][0]]
                fourth.rec  = [batch['REC::Particle_vz'][i][kk]] 
                #fourth.mom = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                fourth.elc = [batch['REC::Particle_vz'][i][0] ] 
                #fourth.rec = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                #idkkk+=[[batch['REC::Particle_vz'][i][kk], batch['REC::Particle_vz'][i][j]]]
                #invarient masss#######################################################################################################
                fourth.imass = [batch['REC::Kinematics_mass'][i][0]] 
                if kk in rec_indices and has_decay == True:
                    fourth.LamT = [1]
                else :
                    fourth.LamT = [0]
                third+= [fourth]
    kkkk+=1
    print(kkkk) 
ii=2 
if j==0:
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_'+str(ii)+'.hipo'
if j==1:
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3316_'+str(ii)+'.hipo'
if j==2:
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3317_'+str(ii)+'.hipo'
if j==3 :
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3318_'+str(ii)+'.hipo'
if j==4:
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3327_'+str(ii)+'.hipo'

step = 16000  
for batch in hp.iterate([filee],banks=["MC::Lund", "REC::Particle", "REC::Traj", "REC::Kinematics"],step=step): 
   # print(batch.keys()) 
    #batch.keys()
    dic = batch.keys() 
    break   

#for battch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_0.hipo'],banks=["REC::Kinematics"],step=step):
    #print(battch.keys()) 
    #batch.keys()
    #dic = battch.keys() 
   # break  
#
all_keys            = list(batch.keys())
rec_particle_name   = 'REC::Particle'
rec_particle_keys   = get_bank_keys(rec_particle_name,all_keys)
rec_traj_name       = 'REC::Traj'
rec_traj_keys       = get_bank_keys(rec_traj_name,all_keys)
rec_kinematics_name = 'REC::Kinematics' 
rec_kinematics_keys = get_bank_keys(rec_kinematics_name,all_keys)
mc_lund_name        = 'MC::Lund'
mc_lund_keys        = get_bank_keys(mc_lund_name,all_keys) 

# rec_kinematics_keys = ['REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 
#                       'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass']
#fulll = [] 
tst = []
y=[]


#j is proton, kk is pion


kinn = []  
idkkk =[]
for i in range(step):
    # Get REC::Particle bank
    rec_particle_event_table = get_event_table(rec_particle_keys,i,batch,dtype=float)

    # Get REC::Traj bank
    rec_traj_event_table = get_event_table(rec_traj_keys,i,batch,dtype=float)

    # Get REC::Kinematics bank
    rec_kinematics_event_table = get_event_table(rec_kinematics_keys,i,batch,dtype=float)

    # Get MC::Lund bank and MC->REC matching indices
    mc_lund_event_table = get_event_table(mc_lund_keys,i,batch,dtype=float)
    match_indices  = get_match_indices(rec_particle_event_table,mc_lund_event_table)

    decay = [3122,[2212,-211]] 
    has_decay, rec_indices = check_has_decay(rec_particle_event_table,mc_lund_event_table,match_indices,decay,
                                   rec_particle_pid_idx=0,mc_lund_pid_idx=3,mc_lund_parent_idx=4,mc_lund_daughter_idx=5)

    #print(rec_indices)
    #getting the index of where the electron proton and pion r 
    savee = batch['REC::Particle_pid'][i]
    j = savee.index(2212); kk = savee.index(-211); elc = savee.index(11) #elc is electron, used for checking that it matches up, j is proton, kk is pionn 
    #ckeck
    #making sure that it is not bad (for the match) #if is bad then lots more stuff to do 
    if match_indices[kk][1]!=-1: 
        #if (np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)<1):
        #if kk in rec_indices and has_decay == True:     
        track_hit = []
        indexp = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == j]
        indexpi = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == kk]
        #get all the data we need here. 

        totalpos =[ [batch['REC::Traj_x'][i][ii] for ii in indexpi], [ batch['REC::Traj_y'][i][ii] for ii in indexpi], [ batch['REC::Traj_z'][i][ii] for ii in indexpi],
                  [batch['REC::Traj_cx'][i][ii] for ii in indexpi], [ batch['REC::Traj_cy'][i][ii] for ii in indexpi], [ batch['REC::Traj_cz'][i][ii] for ii in indexpi]]


        s = 0
        ss =0 
        for dd in range(len(indexpi)):
            track_hit += [[batch['REC::Traj_x'][i][indexpi[dd]], batch['REC::Traj_y'][i][indexpi[dd]], batch['REC::Traj_z'][i][indexpi[dd]],
                         batch['REC::Traj_cx'][i][indexpi[dd]], batch['REC::Traj_cy'][i][indexpi[dd]], batch['REC::Traj_cz'][i][indexpi[dd]],
                           batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                         # batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk],
                           #random.gauss(batch['REC::Particle_px'][i][kk], 1), random.gauss(batch['REC::Particle_py'][i][kk], 1), 
                           #random.gauss(batch['REC::Particle_pz'][i][kk], 1),
                           random.gauss(batch['REC::Particle_vx'][i][kk], 3.394), random.gauss(batch['REC::Particle_vy'][i][kk], 3.394), 
                           random.gauss(batch['REC::Particle_vz'][i][kk], 3.394)]]
            s +=1
        #track_hit+=[[batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
         #           batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk]]]
        ss =0 
        for dd in range(len(indexp)): 
            track_hit += [[batch['REC::Traj_x'][i][indexp[dd]], batch['REC::Traj_y'][i][indexp[dd]], batch['REC::Traj_z'][i][indexp[dd]],
                           batch['REC::Traj_cx'][i][indexp[dd]], batch['REC::Traj_cy'][i][indexp[dd]], batch['REC::Traj_cz'][i][indexp[dd]],
                          batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j],
                          #batch['REC::Particle_vx'][i][j], batch['REC::Particle_vy'][i][j], batch['REC::Particle_vz'][i][j],
                           #random.gauss(batch['REC::Particle_px'][i][j], 1), random.gauss(batch['REC::Particle_py'][i][j], 1), 
                           #random.gauss(batch['REC::Particle_pz'][i][j], 1),
                           random.gauss(batch['REC::Particle_vx'][i][j], 3.394), random.gauss(batch['REC::Particle_vy'][i][j], 3.394), 
                           random.gauss(batch['REC::Particle_vz'][i][j], 3.394)]]

            ss+=1  


        y = [batch['MC::Lund_vz'][i][match_indices[kk][1]] ] 
        #y = [np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)]  

        #track_hit += [[np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)]]

        start = []
        end = []
        for gg in range(s-1):
            start.append(gg)
            end.append(gg+1)
        for ggg in range(ss-1): 
            start.append(s+ggg) 
            end.append(s+ggg+1)
        #fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
        fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
       # fourth.kinematics = [battch['REC::Kinematics_Q2'][i][0], battch['REC::Kinematics_nu'][i][0], battch['REC::Kinematics_W'][i][0], battch['REC::Kinematics_x'][i][0],  
        #                     battch['REC::Kinematics_y'][i][0], battch['REC::Kinematics_z'][i][0], battch['REC::Kinematics_xF'][i][0], battch['REC::Kinematics_mass'][i][0]]
        fourth.rec  = [batch['REC::Particle_vz'][i][kk]] 
        #fourth.mom = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
        fourth.elc = [batch['REC::Particle_vz'][i][0] ] 
        #fourth.rec = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
        #idkkk+=[[batch['REC::Particle_vz'][i][kk], batch['REC::Particle_vz'][i][j]]]
        #invarient masss#######################################################################################################
        fourth.imass = [batch['REC::Kinematics_mass'][i][0]] 
        if kk in rec_indices and has_decay == True:
            fourth.LamT = [1]
        else :
            fourth.LamT = [0]
        third+= [fourth]
kkkk+=1
print(kkkk) 

data_list = third  
root = '/work/clas12/users/mencke/QuatorL_Last'        
class MyDataset(InMemoryDataset):
    def __init__(self, root, data_list, transform=None): 
        self.data_list = data_list
        super().__init__(root, transform)  
        self.load(self.processed_paths[0])

    @property
    def processed_file_names(self): 
        return 'data.pt' 
 
    def process(self):
        self.save(self.data_list, self.processed_paths[0])     

MyDataset(root, third)      

1
2


Processing...
Done!


MyDataset(18500)

In [ ]:
root = '/work/clas12/users/mencke/HalfL_first'  j=4
/work/clas12/users/mencke/HalfL_second' , j=0
/work/clas12/users/mencke/HalfL_third' , j=1 
/work/clas12/users/mencke/HalfL_fourth' , j=2
/work/clas12/users/mencke/HalfL_last' , j=3
root = '/work/clas12/users/mencke/HalfLL_last', j =3
'/work/clas12/users/mencke/HalfL_fifth, j=3

#now this is quator datasets. 
ork/clas12/users/mencke/QuatorL_first', j=0 
'/work/clas12/users/mencke/QuatorL_second', j=1
root = '/work/clas12/users/mencke/QuatorL_third', j=2
root = '/work/clas12/users/mencke/QuatorL_fourth', j=3
ork/clas12/users/mencke/QuatorL_fifth, j=4 
root = '/work/clas12/users/mencke/QuatorL_Last' , j=4   

In [10]:
#Making pure lambda datasets of size 18k 
from torch_geometric.data import Dataset, download_url, InMemoryDataset

 #third is list of graphss  
third = []
kkkk = 0
#for j in range(1):    
 #   number = 5
    #if j==2:
      #  number = 2
j=4 
number = 5   
for ii in range(number):   
    if j==0:
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_'+str(ii)+'.hipo'
    if j==1:
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3316_'+str(ii)+'.hipo'
    if j==2:
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3317_'+str(ii)+'.hipo'
    if j==3 :
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3318_'+str(ii)+'.hipo'
    if j==4:
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3327_'+str(ii)+'.hipo'

    step = 40000
    for batch in hp.iterate([filee],banks=["MC::Lund", "REC::Particle", "REC::Traj", "REC::Kinematics"],step=step): 
       # print(batch.keys()) 
        #batch.keys()
        dic = batch.keys() 
        break   

    #for battch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_0.hipo'],banks=["REC::Kinematics"],step=step):
        #print(battch.keys()) 
        #batch.keys()
        #dic = battch.keys() 
       # break  
    #
    all_keys            = list(batch.keys())
    rec_particle_name   = 'REC::Particle'
    rec_particle_keys   = get_bank_keys(rec_particle_name,all_keys)
    rec_traj_name       = 'REC::Traj'
    rec_traj_keys       = get_bank_keys(rec_traj_name,all_keys)
    rec_kinematics_name = 'REC::Kinematics' 
    rec_kinematics_keys = get_bank_keys(rec_kinematics_name,all_keys)
    mc_lund_name        = 'MC::Lund'
    mc_lund_keys        = get_bank_keys(mc_lund_name,all_keys) 

    # rec_kinematics_keys = ['REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 
    #                       'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass']
    #fulll = [] 
    tst = []
    y=[]


    #j is proton, kk is pion


    kinn = []  
    idkkk =[]
    for i in range(step):
        # Get REC::Particle bank
        rec_particle_event_table = get_event_table(rec_particle_keys,i,batch,dtype=float)

        # Get REC::Traj bank
        rec_traj_event_table = get_event_table(rec_traj_keys,i,batch,dtype=float)

        # Get REC::Kinematics bank
        rec_kinematics_event_table = get_event_table(rec_kinematics_keys,i,batch,dtype=float)

        # Get MC::Lund bank and MC->REC matching indices
        mc_lund_event_table = get_event_table(mc_lund_keys,i,batch,dtype=float)
        match_indices  = get_match_indices(rec_particle_event_table,mc_lund_event_table)

        decay = [3122,[2212,-211]] 
        has_decay, rec_indices = check_has_decay(rec_particle_event_table,mc_lund_event_table,match_indices,decay,
                                       rec_particle_pid_idx=0,mc_lund_pid_idx=3,mc_lund_parent_idx=4,mc_lund_daughter_idx=5)

        #print(rec_indices)
        #getting the index of where the electron proton and pion r 
        savee = batch['REC::Particle_pid'][i]
        j = savee.index(2212); kk = savee.index(-211); elc = savee.index(11) #elc is electron, used for checking that it matches up, j is proton, kk is pionn 
        #ckeck
        #making sure that it is not bad (for the match) #if is bad then lots more stuff to do 
        if match_indices[kk][1]!=-1: 
            #if (np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)<1):
            if kk in rec_indices and has_decay == True:    
                track_hit = []
                indexp = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == j]
                indexpi = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == kk]
                #get all the data we need here. 

                totalpos =[ [batch['REC::Traj_x'][i][ii] for ii in indexpi], [ batch['REC::Traj_y'][i][ii] for ii in indexpi], [ batch['REC::Traj_z'][i][ii] for ii in indexpi],
                          [batch['REC::Traj_cx'][i][ii] for ii in indexpi], [ batch['REC::Traj_cy'][i][ii] for ii in indexpi], [ batch['REC::Traj_cz'][i][ii] for ii in indexpi]]


                s = 0
                ss =0 
                for dd in range(len(indexpi)):
                    track_hit += [[batch['REC::Traj_x'][i][indexpi[dd]], batch['REC::Traj_y'][i][indexpi[dd]], batch['REC::Traj_z'][i][indexpi[dd]],
                                 batch['REC::Traj_cx'][i][indexpi[dd]], batch['REC::Traj_cy'][i][indexpi[dd]], batch['REC::Traj_cz'][i][indexpi[dd]],
                                   batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                                 # batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk],
                                   #random.gauss(batch['REC::Particle_px'][i][kk], 1), random.gauss(batch['REC::Particle_py'][i][kk], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][kk], 1),
                                   random.gauss(batch['REC::Particle_vx'][i][kk], 3.394), random.gauss(batch['REC::Particle_vy'][i][kk], 3.394), 
                                   random.gauss(batch['REC::Particle_vz'][i][kk], 3.394)]]
                    s +=1
                #track_hit+=[[batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                 #           batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk]]]
                ss =0 
                for dd in range(len(indexp)): 
                    track_hit += [[batch['REC::Traj_x'][i][indexp[dd]], batch['REC::Traj_y'][i][indexp[dd]], batch['REC::Traj_z'][i][indexp[dd]],
                                   batch['REC::Traj_cx'][i][indexp[dd]], batch['REC::Traj_cy'][i][indexp[dd]], batch['REC::Traj_cz'][i][indexp[dd]],
                                  batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j],
                                  #batch['REC::Particle_vx'][i][j], batch['REC::Particle_vy'][i][j], batch['REC::Particle_vz'][i][j],
                                   #random.gauss(batch['REC::Particle_px'][i][j], 1), random.gauss(batch['REC::Particle_py'][i][j], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][j], 1),
                                   random.gauss(batch['REC::Particle_vx'][i][j], 3.394), random.gauss(batch['REC::Particle_vy'][i][j], 3.394), 
                                   random.gauss(batch['REC::Particle_vz'][i][j], 3.394)]]

                    ss+=1  


                y = [batch['MC::Lund_vz'][i][match_indices[kk][1]] ] 
                #y = [np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)]  

                #track_hit += [[np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)]]

                start = []
                end = []
                for gg in range(s-1):
                    start.append(gg)
                    end.append(gg+1)
                for ggg in range(ss-1): 
                    start.append(s+ggg) 
                    end.append(s+ggg+1)
                #fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
                fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
               # fourth.kinematics = [battch['REC::Kinematics_Q2'][i][0], battch['REC::Kinematics_nu'][i][0], battch['REC::Kinematics_W'][i][0], battch['REC::Kinematics_x'][i][0],  
                #                     battch['REC::Kinematics_y'][i][0], battch['REC::Kinematics_z'][i][0], battch['REC::Kinematics_xF'][i][0], battch['REC::Kinematics_mass'][i][0]]
                fourth.rec  = [batch['REC::Particle_vz'][i][kk]] 
                #fourth.mom = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                fourth.elc = [batch['REC::Particle_vz'][i][0] ] 
                #fourth.rec = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                #idkkk+=[[batch['REC::Particle_vz'][i][kk], batch['REC::Particle_vz'][i][j]]]
                #invarient masss#######################################################################################################
                fourth.imass = [batch['REC::Kinematics_mass'][i][0]] 
                if kk in rec_indices and has_decay == True:
                    fourth.LamT = [1]
                else :
                    fourth.LamT = [0]
                third+= [fourth]
    kkkk+=1
    print(kkkk) 
data_list = third   
root = '/work/clas12/users/mencke/Lambda_fith'       
class MyDataset(InMemoryDataset):
    def __init__(self, root, data_list, transform=None): 
        self.data_list = data_list
        super().__init__(root, transform) 
        self.load(self.processed_paths[0])

    @property
    def processed_file_names(self):
        return 'data.pt' 
 
    def process(self):
        self.save(self.data_list, self.processed_paths[0])     

MyDataset(root, third)   

1
2
3
4
5


Processing...
Done!


MyDataset(18906)

In [ ]:
'/work/clas12/users/mencke/Lambda_first' , j=0 
'/work/clas12/users/mencke/Lambda_second' , j=1 
'/work/clas12/users/mencke/Lambda_third' , j=2 18944
'/work/clas12/users/mencke/Lambda_fourth', j=3 
work/clas12/users/mencke/Lambda_fith'  , j=4 

In [5]:
from torch_geometric.data import Dataset, download_url, InMemoryDataset

 #third is list of graphss   
third = []
kkkk = 0
for j in range(5):    
    number = 5
    #if j==2:
      #  number = 2
    for ii in range(number):  
        if j==0:
            filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_'+str(ii)+'.hipo'
        if j==1:
            filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3316_'+str(ii)+'.hipo'
        if j==2:
            filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3317_'+str(ii)+'.hipo'
        if j==3 :
            filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3318_'+str(ii)+'.hipo'
        if j==4:
            filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3327_'+str(ii)+'.hipo'
        
        step = 40000 
        for batch in hp.iterate([filee],banks=["MC::Lund", "REC::Particle", "REC::Traj", "REC::Kinematics"],step=step): 
           # print(batch.keys()) 
            #batch.keys()
            dic = batch.keys() 
            break   

        #for battch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_0.hipo'],banks=["REC::Kinematics"],step=step):
            #print(battch.keys()) 
            #batch.keys()
            #dic = battch.keys() 
           # break  
        #
        all_keys            = list(batch.keys())
        rec_particle_name   = 'REC::Particle'
        rec_particle_keys   = get_bank_keys(rec_particle_name,all_keys)
        rec_traj_name       = 'REC::Traj'
        rec_traj_keys       = get_bank_keys(rec_traj_name,all_keys)
        rec_kinematics_name = 'REC::Kinematics' 
        rec_kinematics_keys = get_bank_keys(rec_kinematics_name,all_keys)
        mc_lund_name        = 'MC::Lund'
        mc_lund_keys        = get_bank_keys(mc_lund_name,all_keys) 

        # rec_kinematics_keys = ['REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 
        #                       'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass']
        #fulll = [] 
        tst = []
        y=[]


        #j is proton, kk is pion


        kinn = []  
        idkkk =[]
        for i in range(step):
            # Get REC::Particle bank
            rec_particle_event_table = get_event_table(rec_particle_keys,i,batch,dtype=float)

            # Get REC::Traj bank
            rec_traj_event_table = get_event_table(rec_traj_keys,i,batch,dtype=float)

            # Get REC::Kinematics bank
            rec_kinematics_event_table = get_event_table(rec_kinematics_keys,i,batch,dtype=float)

            # Get MC::Lund bank and MC->REC matching indices
            mc_lund_event_table = get_event_table(mc_lund_keys,i,batch,dtype=float)
            match_indices  = get_match_indices(rec_particle_event_table,mc_lund_event_table)

            decay = [3122,[2212,-211]] 
            has_decay, rec_indices = check_has_decay(rec_particle_event_table,mc_lund_event_table,match_indices,decay,
                                           rec_particle_pid_idx=0,mc_lund_pid_idx=3,mc_lund_parent_idx=4,mc_lund_daughter_idx=5)

            #print(rec_indices)
            #getting the index of where the electron proton and pion r 
            savee = batch['REC::Particle_pid'][i]
            j = savee.index(2212); kk = savee.index(-211); elc = savee.index(11) #elc is electron, used for checking that it matches up, j is proton, kk is pionn 
            #ckeck
            #making sure that it is not bad (for the match) #if is bad then lots more stuff to do 
            if match_indices[kk][1]!=-1: 
                #if (np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)<1):
                #if kk in rec_indices and has_decay == True:    
                track_hit = []
                indexp = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == j]
                indexpi = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == kk]
                #get all the data we need here. 

                totalpos =[ [batch['REC::Traj_x'][i][ii] for ii in indexpi], [ batch['REC::Traj_y'][i][ii] for ii in indexpi], [ batch['REC::Traj_z'][i][ii] for ii in indexpi],
                          [batch['REC::Traj_cx'][i][ii] for ii in indexpi], [ batch['REC::Traj_cy'][i][ii] for ii in indexpi], [ batch['REC::Traj_cz'][i][ii] for ii in indexpi]]


                s = 0
                ss =0 
                for dd in range(len(indexpi)):
                    track_hit += [[batch['REC::Traj_x'][i][indexpi[dd]], batch['REC::Traj_y'][i][indexpi[dd]], batch['REC::Traj_z'][i][indexpi[dd]],
                                 batch['REC::Traj_cx'][i][indexpi[dd]], batch['REC::Traj_cy'][i][indexpi[dd]], batch['REC::Traj_cz'][i][indexpi[dd]],
                                   batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                                 # batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk],
                                   #random.gauss(batch['REC::Particle_px'][i][kk], 1), random.gauss(batch['REC::Particle_py'][i][kk], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][kk], 1),
                                   random.gauss(batch['REC::Particle_vx'][i][kk], 3.394), random.gauss(batch['REC::Particle_vy'][i][kk], 3.394), 
                                   random.gauss(batch['REC::Particle_vz'][i][kk], 3.394)]]
                    s +=1
                #track_hit+=[[batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                 #           batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk]]]
                ss =0 
                for dd in range(len(indexp)): 
                    track_hit += [[batch['REC::Traj_x'][i][indexp[dd]], batch['REC::Traj_y'][i][indexp[dd]], batch['REC::Traj_z'][i][indexp[dd]],
                                   batch['REC::Traj_cx'][i][indexp[dd]], batch['REC::Traj_cy'][i][indexp[dd]], batch['REC::Traj_cz'][i][indexp[dd]],
                                  batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j],
                                  #batch['REC::Particle_vx'][i][j], batch['REC::Particle_vy'][i][j], batch['REC::Particle_vz'][i][j],
                                   #random.gauss(batch['REC::Particle_px'][i][j], 1), random.gauss(batch['REC::Particle_py'][i][j], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][j], 1),
                                   random.gauss(batch['REC::Particle_vx'][i][j], 3.394), random.gauss(batch['REC::Particle_vy'][i][j], 3.394), 
                                   random.gauss(batch['REC::Particle_vz'][i][j], 3.394)]]

                    ss+=1  


                y = [batch['MC::Lund_vz'][i][match_indices[kk][1]] ] 
                #y = [np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)]  

                #track_hit += [[np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)]]

                start = []
                end = []
                for gg in range(s-1):
                    start.append(gg)
                    end.append(gg+1)
                for ggg in range(ss-1): 
                    start.append(s+ggg) 
                    end.append(s+ggg+1)
                #fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
                fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
               # fourth.kinematics = [battch['REC::Kinematics_Q2'][i][0], battch['REC::Kinematics_nu'][i][0], battch['REC::Kinematics_W'][i][0], battch['REC::Kinematics_x'][i][0],  
                #                     battch['REC::Kinematics_y'][i][0], battch['REC::Kinematics_z'][i][0], battch['REC::Kinematics_xF'][i][0], battch['REC::Kinematics_mass'][i][0]]
                fourth.rec  = [batch['REC::Particle_vz'][i][kk]] 
                #fourth.mom = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                fourth.elc = [batch['REC::Particle_vz'][i][0] ] 
                #fourth.rec = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                #idkkk+=[[batch['REC::Particle_vz'][i][kk], batch['REC::Particle_vz'][i][j]]]
                #invarient masss#######################################################################################################
                fourth.imass = [batch['REC::Kinematics_mass'][i][0]] 
                if kk in rec_indices and has_decay == True:
                    fourth.LamT = [1]
                else :
                    fourth.LamT = [0]
                third+= [fourth]
        kkkk+=1
        print(kkkk) 
data_list = third  
root = '/work/clas12/users/mencke/everythingg'     
class MyDataset(InMemoryDataset):
    def __init__(self, root, data_list, transform=None): 
        self.data_list = data_list
        super().__init__(root, transform) 
        self.load(self.processed_paths[0])

    @property
    def processed_file_names(self):
        return 'data.pt'
 
    def process(self):
        self.save(self.data_list, self.processed_paths[0])     

MyDataset(root, third)   


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


Processing...
Done!


MyDataset(924553)

In [ ]:
root = '/work/clas12/users/mencke/Lambda_100k' all the lambda

In [ ]:
break 
#####################################################################
filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3317_'+str(0)+'.hipo'
        
for batch in hp.iterate([filee],banks=["MC::Lund", "REC::Particle", "REC::Traj", "REC::Kinematics"],step=step): 
       # print(batch.keys()) 
        #batch.keys()
        dic = batch.keys() 
        break   

#for battch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_0.hipo'],banks=["REC::Kinematics"],step=step):
    #print(battch.keys()) 
    #batch.keys()
    #dic = battch.keys() 
   # break  
#
all_keys            = list(batch.keys())
rec_particle_name   = 'REC::Particle'
rec_particle_keys   = get_bank_keys(rec_particle_name,all_keys)
rec_traj_name       = 'REC::Traj'
rec_traj_keys       = get_bank_keys(rec_traj_name,all_keys)
rec_kinematics_name = 'REC::Kinematics' 
rec_kinematics_keys = get_bank_keys(rec_kinematics_name,all_keys)
mc_lund_name        = 'MC::Lund'
mc_lund_keys        = get_bank_keys(mc_lund_name,all_keys) 

# rec_kinematics_keys = ['REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 
#                       'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass']
#fulll = [] 
tst = []
y=[]


#j is proton, kk is pion


kinn = []  
idkkk =[]
for i in range(step):
    # Get REC::Particle bank
    rec_particle_event_table = get_event_table(rec_particle_keys,i,batch,dtype=float)

    # Get REC::Traj bank
    rec_traj_event_table = get_event_table(rec_traj_keys,i,batch,dtype=float)

    # Get REC::Kinematics bank
    rec_kinematics_event_table = get_event_table(rec_kinematics_keys,i,batch,dtype=float)

    # Get MC::Lund bank and MC->REC matching indices
    mc_lund_event_table = get_event_table(mc_lund_keys,i,batch,dtype=float)
    match_indices  = get_match_indices(rec_particle_event_table,mc_lund_event_table)

    decay = [3122,[2212,-211]] 
    has_decay, rec_indices = check_has_decay(rec_particle_event_table,mc_lund_event_table,match_indices,decay,
                                   rec_particle_pid_idx=0,mc_lund_pid_idx=3,mc_lund_parent_idx=4,mc_lund_daughter_idx=5)

    #print(rec_indices)
    #getting the index of where the electron proton and pion r 
    savee = batch['REC::Particle_pid'][i]
    j = savee.index(2212); kk = savee.index(-211); elc = savee.index(11) #elc is electron, used for checking that it matches up, j is proton, kk is pionn 
    #ckeck
    #making sure that it is not bad (for the match) #if is bad then lots more stuff to do 
    if match_indices[kk][1]!=-1: 
        #if (np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)<1):
        #if kk in rec_indices and has_decay == True:    
        track_hit = []
        indexp = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == j]
        indexpi = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == kk]
        #get all the data we need here. 

        totalpos =[ [batch['REC::Traj_x'][i][ii] for ii in indexpi], [ batch['REC::Traj_y'][i][ii] for ii in indexpi], [ batch['REC::Traj_z'][i][ii] for ii in indexpi],
                  [batch['REC::Traj_cx'][i][ii] for ii in indexpi], [ batch['REC::Traj_cy'][i][ii] for ii in indexpi], [ batch['REC::Traj_cz'][i][ii] for ii in indexpi]]


        s = 0
        ss =0 
        for dd in range(len(indexpi)):
            track_hit += [[batch['REC::Traj_x'][i][indexpi[dd]], batch['REC::Traj_y'][i][indexpi[dd]], batch['REC::Traj_z'][i][indexpi[dd]],
                         batch['REC::Traj_cx'][i][indexpi[dd]], batch['REC::Traj_cy'][i][indexpi[dd]], batch['REC::Traj_cz'][i][indexpi[dd]],
                           batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                         # batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk],
                           #random.gauss(batch['REC::Particle_px'][i][kk], 1), random.gauss(batch['REC::Particle_py'][i][kk], 1), 
                           #random.gauss(batch['REC::Particle_pz'][i][kk], 1),
                           random.gauss(batch['REC::Particle_vx'][i][kk], 3.394), random.gauss(batch['REC::Particle_vy'][i][kk], 3.394), 
                           random.gauss(batch['REC::Particle_vz'][i][kk], 3.394)]]
            s +=1
        #track_hit+=[[batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
         #           batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk]]]
        ss =0 
        for dd in range(len(indexp)): 
            track_hit += [[batch['REC::Traj_x'][i][indexp[dd]], batch['REC::Traj_y'][i][indexp[dd]], batch['REC::Traj_z'][i][indexp[dd]],
                           batch['REC::Traj_cx'][i][indexp[dd]], batch['REC::Traj_cy'][i][indexp[dd]], batch['REC::Traj_cz'][i][indexp[dd]],
                          batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j],
                          #batch['REC::Particle_vx'][i][j], batch['REC::Particle_vy'][i][j], batch['REC::Particle_vz'][i][j],
                           #random.gauss(batch['REC::Particle_px'][i][j], 1), random.gauss(batch['REC::Particle_py'][i][j], 1), 
                           #random.gauss(batch['REC::Particle_pz'][i][j], 1),
                           random.gauss(batch['REC::Particle_vx'][i][j], 3.394), random.gauss(batch['REC::Particle_vy'][i][j], 3.394), 
                           random.gauss(batch['REC::Particle_vz'][i][j], 3.394)]]

            ss+=1  


        y = [batch['MC::Lund_vz'][i][match_indices[kk][1]] ] 
        #y = [np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)]  

        #track_hit += [[np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)]]

        start = []
        end = []
        for gg in range(s-1):
            start.append(gg)
            end.append(gg+1)
        for ggg in range(ss-1): 
            start.append(s+ggg) 
            end.append(s+ggg+1)
        #fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
        fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
       # fourth.kinematics = [battch['REC::Kinematics_Q2'][i][0], battch['REC::Kinematics_nu'][i][0], battch['REC::Kinematics_W'][i][0], battch['REC::Kinematics_x'][i][0],  
        #                     battch['REC::Kinematics_y'][i][0], battch['REC::Kinematics_z'][i][0], battch['REC::Kinematics_xF'][i][0], battch['REC::Kinematics_mass'][i][0]]
        fourth.rec  = [batch['REC::Particle_vz'][i][kk]] 
        #fourth.mom = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
        fourth.elc = [batch['REC::Particle_vz'][i][0] ] 
        #fourth.rec = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
        #idkkk+=[[batch['REC::Particle_vz'][i][kk], batch['REC::Particle_vz'][i][j]]]
        #invarient masss#######################################################################################################
        fourth.imass = [batch['REC::Kinematics_mass'][i][0]] 
        if kk in rec_indices and has_decay == True:
            fourth.LamT = [1]
        else :
            fourth.LamT = [0]
        third+= [fourth]
kkkk+=1
print(kkkk) 
data_list = third  
root = '/work/clas12/users/mencke/halfL_halff'     
class MyDataset(InMemoryDataset):
    def __init__(self, root, data_list, transform=None): 
        self.data_list = data_list
        super().__init__(root, transform) 
        self.load(self.processed_paths[0])

    @property
    def processed_file_names(self):
        return 'data.pt'
 
    def process(self):
        self.save(self.data_list, self.processed_paths[0])     

MyDataset(root, third)    
#root = '/work/clas12/users/mencke/Lambda_13_16'  all 5 from 3313 and 3316 
#root = '/work/clas12/users/mencke/Lambda_1all' all lambda events 94549 events 
#root = '/work/clas12/users/mencke/halfL_halff', first two sets are only lanbda, then a normal one, should be around half and half. 

In [36]:
from torch_geometric.data import Dataset, download_url, InMemoryDataset   

step = 40000
for batch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_0.hipo'],banks=["MC::Lund", "REC::Particle", "REC::Traj", "REC::Kinematics"],step=step):
    print(batch.keys()) 
    #batch.keys()
    dic = batch.keys() 
    break  
    
#for battch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_0.hipo'],banks=["REC::Kinematics"],step=step):
    #print(battch.keys()) 
    #batch.keys()
    #dic = battch.keys() 
   # break  
#

all_keys            = list(batch.keys())
rec_particle_name   = 'REC::Particle'
rec_particle_keys   = get_bank_keys(rec_particle_name,all_keys)
rec_traj_name       = 'REC::Traj'
rec_traj_keys       = get_bank_keys(rec_traj_name,all_keys)
rec_kinematics_name = 'REC::Kinematics' 
rec_kinematics_keys = get_bank_keys(rec_kinematics_name,all_keys)
mc_lund_name        = 'MC::Lund'
mc_lund_keys        = get_bank_keys(mc_lund_name,all_keys) 

# rec_kinematics_keys = ['REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 
#                       'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass']
#fulll = [] 
tst = []
y=[]
#third is list of graphss 
third = []
#j is proton, kk is pion


kinn = []  
idkkk =[]
for i in range(step):
    # Get REC::Particle bank
    rec_particle_event_table = get_event_table(rec_particle_keys,i,batch,dtype=float)

    # Get REC::Traj bank
    rec_traj_event_table = get_event_table(rec_traj_keys,i,batch,dtype=float)

    # Get REC::Kinematics bank
    rec_kinematics_event_table = get_event_table(rec_kinematics_keys,i,batch,dtype=float)

    # Get MC::Lund bank and MC->REC matching indices
    mc_lund_event_table = get_event_table(mc_lund_keys,i,batch,dtype=float)
    match_indices  = get_match_indices(rec_particle_event_table,mc_lund_event_table)
    
    decay = [3122,[2212,-211]] 
    has_decay, rec_indices = check_has_decay(rec_particle_event_table,mc_lund_event_table,match_indices,decay,
                                   rec_particle_pid_idx=0,mc_lund_pid_idx=3,mc_lund_parent_idx=4,mc_lund_daughter_idx=5)
    
    #print(rec_indices)
    #getting the index of where the electron proton and pion r 
    savee = batch['REC::Particle_pid'][i]
    j = savee.index(2212); kk = savee.index(-211); elc = savee.index(11) #elc is electron, used for checking that it matches up, j is proton, kk is pionn 
    #ckeck
    #making sure that it is not bad (for the match) #if is bad then lots more stuff to do 
    if match_indices[kk][1]!=-1: 
        #if (np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)<1):
            #if kk in rec_indices and has_decay == True: 
        track_hit = []
        indexp = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == j]
        indexpi = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == kk]
        #get all the data we need here. 
 
        totalpos =[ [batch['REC::Traj_x'][i][ii] for ii in indexpi], [ batch['REC::Traj_y'][i][ii] for ii in indexpi], [ batch['REC::Traj_z'][i][ii] for ii in indexpi],
                  [batch['REC::Traj_cx'][i][ii] for ii in indexpi], [ batch['REC::Traj_cy'][i][ii] for ii in indexpi], [ batch['REC::Traj_cz'][i][ii] for ii in indexpi]]


        s = 0
        ss =0 
        for dd in range(len(indexpi)):
            track_hit += [[batch['REC::Traj_x'][i][indexpi[dd]], batch['REC::Traj_y'][i][indexpi[dd]], batch['REC::Traj_z'][i][indexpi[dd]],
                         batch['REC::Traj_cx'][i][indexpi[dd]], batch['REC::Traj_cy'][i][indexpi[dd]], batch['REC::Traj_cz'][i][indexpi[dd]],
                           batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                         # batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk],
                           #random.gauss(batch['REC::Particle_px'][i][kk], 1), random.gauss(batch['REC::Particle_py'][i][kk], 1), 
                           #random.gauss(batch['REC::Particle_pz'][i][kk], 1),
                           random.gauss(batch['REC::Particle_vx'][i][kk], 3.394), random.gauss(batch['REC::Particle_vy'][i][kk], 3.394), 
                           random.gauss(batch['REC::Particle_vz'][i][kk], 3.394)]]
            s +=1
        #track_hit+=[[batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
         #           batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk]]]
        ss =0
        for dd in range(len(indexp)): 
            track_hit += [[batch['REC::Traj_x'][i][indexp[dd]], batch['REC::Traj_y'][i][indexp[dd]], batch['REC::Traj_z'][i][indexp[dd]],
                           batch['REC::Traj_cx'][i][indexp[dd]], batch['REC::Traj_cy'][i][indexp[dd]], batch['REC::Traj_cz'][i][indexp[dd]],
                          batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j],
                          #batch['REC::Particle_vx'][i][j], batch['REC::Particle_vy'][i][j], batch['REC::Particle_vz'][i][j],
                           #random.gauss(batch['REC::Particle_px'][i][j], 1), random.gauss(batch['REC::Particle_py'][i][j], 1), 
                           #random.gauss(batch['REC::Particle_pz'][i][j], 1),
                           random.gauss(batch['REC::Particle_vx'][i][j], 3.394), random.gauss(batch['REC::Particle_vy'][i][j], 3.394), 
                           random.gauss(batch['REC::Particle_vz'][i][j], 3.394)]]

            ss+=1  


        y = [batch['MC::Lund_vz'][i][match_indices[kk][1]] ] 
        #y = [np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)]  

        #track_hit += [[np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)]]

        start = []
        end = []
        for gg in range(s-1):
            start.append(gg)
            end.append(gg+1)
        for ggg in range(ss-1): 
            start.append(s+ggg)
            end.append(s+ggg+1)
        #fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
        fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
       # fourth.kinematics = [battch['REC::Kinematics_Q2'][i][0], battch['REC::Kinematics_nu'][i][0], battch['REC::Kinematics_W'][i][0], battch['REC::Kinematics_x'][i][0],  
        #                     battch['REC::Kinematics_y'][i][0], battch['REC::Kinematics_z'][i][0], battch['REC::Kinematics_xF'][i][0], battch['REC::Kinematics_mass'][i][0]]
        fourth.rec  = [batch['REC::Particle_vz'][i][kk]] 
        #fourth.mom = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
        fourth.elc = [batch['REC::Particle_vz'][i][0] ] 
        #fourth.rec = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
        #idkkk+=[[batch['REC::Particle_vz'][i][kk], batch['REC::Particle_vz'][i][j]]]
        #invarient masss#######################################################################################################
        fourth.imass = [batch['REC::Kinematics_mass'][i][0]] 
        third+= [fourth] 
data_list = third 
root = '/work/clas12/users/mencke/big_testt'     
class MyDataset(InMemoryDataset):
    def __init__(self, root, data_list, transform=None): 
        self.data_list = data_list
        super().__init__(root, transform) 
        self.load(self.processed_paths[0])

    @property
    def processed_file_names(self):
        return 'data.pt'
 
    def process(self):
        self.save(self.data_list, self.processed_paths[0])    

MyDataset(root, third)  

dict_keys(['MC::Lund_index', 'MC::Lund_lifetime', 'MC::Lund_type', 'MC::Lund_pid', 'MC::Lund_parent', 'MC::Lund_daughter', 'MC::Lund_px', 'MC::Lund_py', 'MC::Lund_pz', 'MC::Lund_energy', 'MC::Lund_mass', 'MC::Lund_vx', 'MC::Lund_vy', 'MC::Lund_vz', 'REC::Particle_pid', 'REC::Particle_px', 'REC::Particle_py', 'REC::Particle_pz', 'REC::Particle_vx', 'REC::Particle_vy', 'REC::Particle_vz', 'REC::Particle_vt', 'REC::Particle_charge', 'REC::Particle_beta', 'REC::Particle_chi2pid', 'REC::Particle_status', 'REC::Traj_pindex', 'REC::Traj_index', 'REC::Traj_detector', 'REC::Traj_layer', 'REC::Traj_x', 'REC::Traj_y', 'REC::Traj_z', 'REC::Traj_cx', 'REC::Traj_cy', 'REC::Traj_cz', 'REC::Traj_path', 'REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass'])


Processing...
Done!


MyDataset(37127)

In [12]:
from torch_geometric.data import Dataset, download_url, InMemoryDataset   

step = 40000
for batch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_0.hipo'],banks=["MC::Lund", "REC::Particle", "REC::Traj", "REC::Kinematics"],step=step):
    print(batch.keys()) 
    #batch.keys()
    dic = batch.keys() 
    break  
    
#for battch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_0.hipo'],banks=["REC::Kinematics"],step=step):
    #print(battch.keys()) 
    #batch.keys()
    #dic = battch.keys() 
   # break  
#

all_keys            = list(batch.keys())
rec_particle_name   = 'REC::Particle'
rec_particle_keys   = get_bank_keys(rec_particle_name,all_keys)
rec_traj_name       = 'REC::Traj'
rec_traj_keys       = get_bank_keys(rec_traj_name,all_keys)
rec_kinematics_name = 'REC::Kinematics' 
rec_kinematics_keys = get_bank_keys(rec_kinematics_name,all_keys)
mc_lund_name        = 'MC::Lund'
mc_lund_keys        = get_bank_keys(mc_lund_name,all_keys) 

# rec_kinematics_keys = ['REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 
#                       'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass']
#fulll = [] 
tst = []
y=[]
#third is list of graphss 
third = []
#j is proton, kk is pion


kinn = []  
idkkk =[]
for i in range(step):
    # Get REC::Particle bank
    rec_particle_event_table = get_event_table(rec_particle_keys,i,batch,dtype=float)

    # Get REC::Traj bank
    rec_traj_event_table = get_event_table(rec_traj_keys,i,batch,dtype=float)

    # Get REC::Kinematics bank
    rec_kinematics_event_table = get_event_table(rec_kinematics_keys,i,batch,dtype=float)

    # Get MC::Lund bank and MC->REC matching indices
    mc_lund_event_table = get_event_table(mc_lund_keys,i,batch,dtype=float)
    match_indices  = get_match_indices(rec_particle_event_table,mc_lund_event_table)
    
    decay = [3122,[2212,-211]] 
    has_decay, rec_indices = check_has_decay(rec_particle_event_table,mc_lund_event_table,match_indices,decay,
                                   rec_particle_pid_idx=0,mc_lund_pid_idx=3,mc_lund_parent_idx=4,mc_lund_daughter_idx=5)
    
    #print(rec_indices)
    #getting the index of where the electron proton and pion r 
    savee = batch['REC::Particle_pid'][i]
    j = savee.index(2212); kk = savee.index(-211); elc = savee.index(11) #elc is electron, used for checking that it matches up, j is proton, kk is pionn 
    #ckeck
    #making sure that it is not bad (for the match) #if is bad then lots more stuff to do 
    if match_indices[kk][1]!=-1: 
        if (np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)<1):
            if kk in rec_indices and has_decay == True: 
                track_hit = []
                indexp = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == j]
                indexpi = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == kk]
                #get all the data we need here. 

                totalpos =[ [batch['REC::Traj_x'][i][ii] for ii in indexpi], [ batch['REC::Traj_y'][i][ii] for ii in indexpi], [ batch['REC::Traj_z'][i][ii] for ii in indexpi],
                          [batch['REC::Traj_cx'][i][ii] for ii in indexpi], [ batch['REC::Traj_cy'][i][ii] for ii in indexpi], [ batch['REC::Traj_cz'][i][ii] for ii in indexpi]]


                s = 0
                ss =0 
                for dd in range(len(indexpi)):
                    track_hit += [[batch['REC::Traj_x'][i][indexpi[dd]], batch['REC::Traj_y'][i][indexpi[dd]], batch['REC::Traj_z'][i][indexpi[dd]],
                                 batch['REC::Traj_cx'][i][indexpi[dd]], batch['REC::Traj_cy'][i][indexpi[dd]], batch['REC::Traj_cz'][i][indexpi[dd]],
                                   batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                                 # batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk],
                                   #random.gauss(batch['REC::Particle_px'][i][kk], 1), random.gauss(batch['REC::Particle_py'][i][kk], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][kk], 1),
                                   random.gauss(batch['REC::Particle_vx'][i][kk], 3.394), random.gauss(batch['REC::Particle_vy'][i][kk], 3.394), 
                                   random.gauss(batch['REC::Particle_vz'][i][kk], 3.394)]]
                    s +=1
                #track_hit+=[[batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                 #           batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk]]]
                ss =0
                for dd in range(len(indexp)): 
                    track_hit += [[batch['REC::Traj_x'][i][indexp[dd]], batch['REC::Traj_y'][i][indexp[dd]], batch['REC::Traj_z'][i][indexp[dd]],
                                   batch['REC::Traj_cx'][i][indexp[dd]], batch['REC::Traj_cy'][i][indexp[dd]], batch['REC::Traj_cz'][i][indexp[dd]],
                                  batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j],
                                  #batch['REC::Particle_vx'][i][j], batch['REC::Particle_vy'][i][j], batch['REC::Particle_vz'][i][j],
                                   #random.gauss(batch['REC::Particle_px'][i][j], 1), random.gauss(batch['REC::Particle_py'][i][j], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][j], 1),
                                   random.gauss(batch['REC::Particle_vx'][i][j], 3.394), random.gauss(batch['REC::Particle_vy'][i][j], 3.394), 
                                   random.gauss(batch['REC::Particle_vz'][i][j], 3.394)]]

                    ss+=1  


                y = [batch['MC::Lund_vz'][i][match_indices[kk][1]] ] 
                #y = [np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)]  

                #track_hit += [[np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)]]

                start = []
                end = []
                for gg in range(s-1):
                    start.append(gg)
                    end.append(gg+1)
                for ggg in range(ss-1): 
                    start.append(s+ggg)
                    end.append(s+ggg+1)
                #fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
                fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
               # fourth.kinematics = [battch['REC::Kinematics_Q2'][i][0], battch['REC::Kinematics_nu'][i][0], battch['REC::Kinematics_W'][i][0], battch['REC::Kinematics_x'][i][0],  
                #                     battch['REC::Kinematics_y'][i][0], battch['REC::Kinematics_z'][i][0], battch['REC::Kinematics_xF'][i][0], battch['REC::Kinematics_mass'][i][0]]
                fourth.rec  = [batch['REC::Particle_vz'][i][kk]] 
                #fourth.mom = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                fourth.elc = [batch['REC::Particle_vz'][i][0] ] 
                #fourth.rec = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                #idkkk+=[[batch['REC::Particle_vz'][i][kk], batch['REC::Particle_vz'][i][j]]]
                #invarient masss#######################################################################################################
                fourth.imass = [batch['REC::Kinematics_mass'][i][0]] 
                third+= [fourth]  
            
            '''
            p_pion = np.array([batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk]]) 
            p_proton = np.array([batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j]])
            P_mass = 938 *10**-3 #GeV 
            pi_mass = 139 *10**-3 #GeV 
            E_pion = np.sqrt(pi_mass**2+np.dot(p_pion, p_pion))
            E_proton = np.sqrt(P_mass**2 +np.dot(p_proton, p_proton))
            E_t = (E_pion+E_proton)**2
            P_t = np.dot(p_pion, p_proton)
            fourth.imass = [np.sqrt(E_t-P_t)] '''
            
            
            '''
            p_pion = np.sqrt(batch['REC::Particle_px'][i][kk]**2+ batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)
            p_proton = np.sqrt(batch['REC::Particle_px'][i][j]**2+ batch['REC::Particle_py'][i][j]**2+ batch['REC::Particle_pz'][i][j]**2)
            P_mass = 938 *10**-3 #GeV 
            pi_mass = 139 *10**-3 #GeV  
            E_pion = np.sqrt(pi_mass**2+p_pion**2)
            E_proton  = np.sqrt(P_mass**2 +p_proton **2)
            #E_t = (E_pion+E_proton)**2
            E_t = (pi_mass+P_mass+p_pion+p_proton)**2 
            #E_t = (E_pion**2+E_proton**2)
            #P_t = (p_pion+p_proton)**2
            P_t = (batch['REC::Particle_px'][i][kk]+batch['REC::Particle_px'][i][j])**2+(batch['REC::Particle_py'][i][kk]+batch['REC::Particle_py'][i][j])**2+(batch['REC::Particle_pz'][i][kk]+batch['REC::Particle_pz'][i][j])**2
            fourth.imass = [np.sqrt(E_t-P_t)]   '''
            

step = 45000
for batch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_1.hipo'],banks=["MC::Lund", "REC::Particle", "REC::Traj", "REC::Kinematics"],step=step):
    print(batch.keys()) 
    #batch.keys()
    dic = batch.keys() 
    break  
all_keys            = list(batch.keys())
rec_particle_name   = 'REC::Particle'
rec_particle_keys   = get_bank_keys(rec_particle_name,all_keys)
rec_traj_name       = 'REC::Traj'
rec_traj_keys       = get_bank_keys(rec_traj_name,all_keys)
#rec_kinematics_name = 'REC::Kinematics' 
#rec_kinematics_keys = get_bank_keys(rec_kinematics_name,all_keys)
mc_lund_name        = 'MC::Lund'
mc_lund_keys        = get_bank_keys(mc_lund_name,all_keys) 

# rec_kinematics_keys = ['REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 
#                       'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass']
#fulll = [] 
tst = []
y=[]
kinn = []  
idkkk =[]
for i in range(step):
    # Get REC::Particle bank
    rec_particle_event_table = get_event_table(rec_particle_keys,i,batch,dtype=float)

    # Get REC::Traj bank
    rec_traj_event_table = get_event_table(rec_traj_keys,i,batch,dtype=float)

    # Get REC::Kinematics bank
    rec_kinematics_event_table = get_event_table(rec_kinematics_keys,i,batch,dtype=float) 

    # Get MC::Lund bank and MC->REC matching indices
    mc_lund_event_table = get_event_table(mc_lund_keys,i,batch,dtype=float)
    match_indices  = get_match_indices(rec_particle_event_table,mc_lund_event_table)
    
    decay = [3122,[2212,-211]] 
    has_decay, rec_indices = check_has_decay(rec_particle_event_table,mc_lund_event_table,match_indices,decay,
                                   rec_particle_pid_idx=0,mc_lund_pid_idx=3,mc_lund_parent_idx=4,mc_lund_daughter_idx=5)
    
    #print(rec_indices)
    #getting the index of where the electron proton and pion r 
    savee = batch['REC::Particle_pid'][i]
    j = savee.index(2212); kk = savee.index(-211); elc = savee.index(11) #elc is electron, used for checking that it matches up, j is proton, kk is pionn 
    #ckeck
    #making sure that it is not bad (for the match) #if is bad then lots more stuff to do 
    if match_indices[kk][1]!=-1: 
        if (np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)<1):
            if kk in rec_indices and has_decay == True: 
                track_hit = []
                indexp = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == j]
                indexpi = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == kk]
                #get all the data we need here. 

                totalpos =[ [batch['REC::Traj_x'][i][ii] for ii in indexpi], [ batch['REC::Traj_y'][i][ii] for ii in indexpi], [ batch['REC::Traj_z'][i][ii] for ii in indexpi],
                          [batch['REC::Traj_cx'][i][ii] for ii in indexpi], [ batch['REC::Traj_cy'][i][ii] for ii in indexpi], [ batch['REC::Traj_cz'][i][ii] for ii in indexpi]]


                s = 0
                ss =0 
                for dd in range(len(indexpi)):
                    track_hit += [[batch['REC::Traj_x'][i][indexpi[dd]], batch['REC::Traj_y'][i][indexpi[dd]], batch['REC::Traj_z'][i][indexpi[dd]],
                                 batch['REC::Traj_cx'][i][indexpi[dd]], batch['REC::Traj_cy'][i][indexpi[dd]], batch['REC::Traj_cz'][i][indexpi[dd]],
                                   batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                                 # batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk],
                                   #random.gauss(batch['REC::Particle_px'][i][kk], 1), random.gauss(batch['REC::Particle_py'][i][kk], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][kk], 1),
                                   random.gauss(batch['REC::Particle_vx'][i][kk], 3.394), random.gauss(batch['REC::Particle_vy'][i][kk], 3.394), 
                                   random.gauss(batch['REC::Particle_vz'][i][kk], 3.394)]]
                    s +=1
                #track_hit+=[[batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                 #           batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk]]]
                ss =0
                for dd in range(len(indexp)): 
                    track_hit += [[batch['REC::Traj_x'][i][indexp[dd]], batch['REC::Traj_y'][i][indexp[dd]], batch['REC::Traj_z'][i][indexp[dd]],
                                   batch['REC::Traj_cx'][i][indexp[dd]], batch['REC::Traj_cy'][i][indexp[dd]], batch['REC::Traj_cz'][i][indexp[dd]],
                                  batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j],
                                  #batch['REC::Particle_vx'][i][j], batch['REC::Particle_vy'][i][j], batch['REC::Particle_vz'][i][j],
                                   #random.gauss(batch['REC::Particle_px'][i][j], 1), random.gauss(batch['REC::Particle_py'][i][j], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][j], 1),
                                   random.gauss(batch['REC::Particle_vx'][i][j], 3.394), random.gauss(batch['REC::Particle_vy'][i][j], 3.394), 
                                   random.gauss(batch['REC::Particle_vz'][i][j], 3.394)]]

                    ss+=1 


                y = [batch['MC::Lund_vz'][i][match_indices[kk][1]] ] 
                #y = [np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)]  

                #track_hit += [[np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)]]

                start = []
                end = []
                for gg in range(s-1):
                    start.append(gg)
                    end.append(gg+1)
                for ggg in range(ss-1): 
                    start.append(s+ggg)
                    end.append(s+ggg+1)
                #fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
                fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
               # fourth.kinematics = [battch['REC::Kinematics_Q2'][i][0], battch['REC::Kinematics_nu'][i][0], battch['REC::Kinematics_W'][i][0], battch['REC::Kinematics_x'][i][0],  
                #                     battch['REC::Kinematics_y'][i][0], battch['REC::Kinematics_z'][i][0], battch['REC::Kinematics_xF'][i][0], battch['REC::Kinematics_mass'][i][0]]
                fourth.rec  = [batch['REC::Particle_vz'][i][kk]] 
                #fourth.mom = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                fourth.elc = [batch['REC::Particle_vz'][i][0] ] 
                #fourth.rec = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                #idkkk+=[[batch['REC::Particle_vz'][i][kk], batch['REC::Particle_vz'][i][j]]]
                #invarient masss####################################################################################################### 
                fourth.imass = [batch['REC::Kinematics_mass'][i][0]]

                '''
                p_pion = np.array([batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk]]) 
                p_proton = np.array([batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j]])
                P_mass = 938 *10**-3 #GeV 
                pi_mass = 139 *10**-3 #GeV 
                E_pion = np.sqrt(pi_mass**2+np.dot(p_pion, p_pion))
                E_proton = np.sqrt(P_mass**2 +np.dot(p_proton, p_proton))
                E_t = (E_pion+E_proton)**2
                P_t = np.dot(p_pion, p_proton)
                fourth.imass = [np.sqrt(E_t-P_t)] '''   


                '''
                p_pion = np.sqrt(batch['REC::Particle_px'][i][kk]**2+ batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)
                p_proton = np.sqrt(batch['REC::Particle_px'][i][j]**2+ batch['REC::Particle_py'][i][j]**2+ batch['REC::Particle_pz'][i][j]**2)
                P_mass = 938 *10**-3 #GeV 
                pi_mass = 139 *10**-3 #GeV  
                E_pion = np.sqrt(pi_mass**2+p_pion**2)
                E_proton  = np.sqrt(P_mass**2 +p_proton **2)
                #E_t = (E_pion+E_proton)**2
                E_t = (pi_mass+P_mass+p_pion+p_proton)**2 
                #E_t = (E_pion**2+E_proton**2)
                #P_t = (p_pion+p_proton)**2
                P_t = (batch['REC::Particle_px'][i][kk]+batch['REC::Particle_px'][i][j])**2+(batch['REC::Particle_py'][i][kk]+batch['REC::Particle_py'][i][j])**2+(batch['REC::Particle_pz'][i][kk]+batch['REC::Particle_pz'][i][j])**2
                fourth.imass = [np.sqrt(E_t-P_t)]   '''


                third+= [fourth]  

step = 45000
for batch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_2.hipo'],banks=["MC::Lund", "REC::Particle", "REC::Traj", "REC::Kinematics"],step=step):
    print(batch.keys()) 
    #batch.keys()
    dic = batch.keys() 
    break  
all_keys            = list(batch.keys())
rec_particle_name   = 'REC::Particle'
rec_particle_keys   = get_bank_keys(rec_particle_name,all_keys)
rec_traj_name       = 'REC::Traj'
rec_traj_keys       = get_bank_keys(rec_traj_name,all_keys)
#rec_kinematics_name = 'REC::Kinematics' 
#rec_kinematics_keys = get_bank_keys(rec_kinematics_name,all_keys)
mc_lund_name        = 'MC::Lund'
mc_lund_keys        = get_bank_keys(mc_lund_name,all_keys) 

# rec_kinematics_keys = ['REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 
#                       'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass']
#fulll = [] 
tst = []
y=[]
kinn = []  
idkkk =[]
for i in range(step):
    # Get REC::Particle bank
    rec_particle_event_table = get_event_table(rec_particle_keys,i,batch,dtype=float)

    # Get REC::Traj bank
    rec_traj_event_table = get_event_table(rec_traj_keys,i,batch,dtype=float)

    # Get REC::Kinematics bank
    rec_kinematics_event_table = get_event_table(rec_kinematics_keys,i,batch,dtype=float)

    # Get MC::Lund bank and MC->REC matching indices
    mc_lund_event_table = get_event_table(mc_lund_keys,i,batch,dtype=float)
    match_indices  = get_match_indices(rec_particle_event_table,mc_lund_event_table)
    
    decay = [3122,[2212,-211]] 
    has_decay, rec_indices = check_has_decay(rec_particle_event_table,mc_lund_event_table,match_indices,decay,
                                   rec_particle_pid_idx=0,mc_lund_pid_idx=3,mc_lund_parent_idx=4,mc_lund_daughter_idx=5)
    
    #print(rec_indices)
    #getting the index of where the electron proton and pion r 
    savee = batch['REC::Particle_pid'][i]
    j = savee.index(2212); kk = savee.index(-211); elc = savee.index(11) #elc is electron, used for checking that it matches up, j is proton, kk is pionn 
    #ckeck
    #making sure that it is not bad (for the match) #if is bad then lots more stuff to do 
    if match_indices[kk][1]!=-1: 
        if (np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)<1):
            if kk in rec_indices and has_decay == True: 
                track_hit = []
                indexp = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == j]
                indexpi = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == kk]
                #get all the data we need here. 

                totalpos =[ [batch['REC::Traj_x'][i][ii] for ii in indexpi], [ batch['REC::Traj_y'][i][ii] for ii in indexpi], [ batch['REC::Traj_z'][i][ii] for ii in indexpi],
                          [batch['REC::Traj_cx'][i][ii] for ii in indexpi], [ batch['REC::Traj_cy'][i][ii] for ii in indexpi], [ batch['REC::Traj_cz'][i][ii] for ii in indexpi]]


                s = 0
                ss =0 
                for dd in range(len(indexpi)):
                    track_hit += [[batch['REC::Traj_x'][i][indexpi[dd]], batch['REC::Traj_y'][i][indexpi[dd]], batch['REC::Traj_z'][i][indexpi[dd]],
                                 batch['REC::Traj_cx'][i][indexpi[dd]], batch['REC::Traj_cy'][i][indexpi[dd]], batch['REC::Traj_cz'][i][indexpi[dd]],
                                   batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                                 # batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk],
                                   #random.gauss(batch['REC::Particle_px'][i][kk], 1), random.gauss(batch['REC::Particle_py'][i][kk], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][kk], 1),
                                   random.gauss(batch['REC::Particle_vx'][i][kk], 3.394), random.gauss(batch['REC::Particle_vy'][i][kk], 3.394), 
                                   random.gauss(batch['REC::Particle_vz'][i][kk], 3.394)]]
                    s +=1
                #track_hit+=[[batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                 #           batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk]]]
                ss =0
                for dd in range(len(indexp)): 
                    track_hit += [[batch['REC::Traj_x'][i][indexp[dd]], batch['REC::Traj_y'][i][indexp[dd]], batch['REC::Traj_z'][i][indexp[dd]],
                                   batch['REC::Traj_cx'][i][indexp[dd]], batch['REC::Traj_cy'][i][indexp[dd]], batch['REC::Traj_cz'][i][indexp[dd]],
                                  batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j],
                                  #batch['REC::Particle_vx'][i][j], batch['REC::Particle_vy'][i][j], batch['REC::Particle_vz'][i][j],
                                   #random.gauss(batch['REC::Particle_px'][i][j], 1), random.gauss(batch['REC::Particle_py'][i][j], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][j], 1),
                                   random.gauss(batch['REC::Particle_vx'][i][j], 3.394), random.gauss(batch['REC::Particle_vy'][i][j], 3.394), 
                                   random.gauss(batch['REC::Particle_vz'][i][j], 3.394)]]

                    ss+=1 


                y = [batch['MC::Lund_vz'][i][match_indices[kk][1]] ] 
                #y = [np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)]  

                #track_hit += [[np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)]]

                start = []
                end = []
                for gg in range(s-1):
                    start.append(gg)
                    end.append(gg+1)
                for ggg in range(ss-1): 
                    start.append(s+ggg) 
                    end.append(s+ggg+1)
                #fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
                fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
               # fourth.kinematics = [battch['REC::Kinematics_Q2'][i][0], battch['REC::Kinematics_nu'][i][0], battch['REC::Kinematics_W'][i][0], battch['REC::Kinematics_x'][i][0],  
                #                     battch['REC::Kinematics_y'][i][0], battch['REC::Kinematics_z'][i][0], battch['REC::Kinematics_xF'][i][0], battch['REC::Kinematics_mass'][i][0]]
                fourth.rec  = [batch['REC::Particle_vz'][i][kk]] 
                #fourth.mom = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                fourth.elc = [batch['REC::Particle_vz'][i][0] ] 
            #fourth.rec = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                #idkkk+=[[batch['REC::Particle_vz'][i][kk], batch['REC::Particle_vz'][i][j]]]
                #invarient masss#######################################################################################################
                fourth.imass = [batch['REC::Kinematics_mass'][i][0]]

                '''
                p_pion = np.array([batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk]]) 
                p_proton = np.array([batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j]])
                P_mass = 938 *10**-3 #GeV 
                pi_mass = 139 *10**-3 #GeV 
                E_pion = np.sqrt(pi_mass**2+np.dot(p_pion, p_pion))
                E_proton = np.sqrt(P_mass**2 +np.dot(p_proton, p_proton))
                E_t = (E_pion+E_proton)**2
                P_t = np.dot(p_pion, p_proton)
                fourth.imass = [np.sqrt(E_t-P_t)] ''' 


                '''
                p_pion = np.sqrt(batch['REC::Particle_px'][i][kk]**2+ batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)
                p_proton = np.sqrt(batch['REC::Particle_px'][i][j]**2+ batch['REC::Particle_py'][i][j]**2+ batch['REC::Particle_pz'][i][j]**2)
                P_mass = 938 *10**-3 #GeV 
                pi_mass = 139 *10**-3 #GeV  
                E_pion = np.sqrt(pi_mass**2+p_pion**2)
                E_proton  = np.sqrt(P_mass**2 +p_proton **2)
                #E_t = (E_pion+E_proton)**2
                E_t = (pi_mass+P_mass+p_pion+p_proton)**2 
                #E_t = (E_pion**2+E_proton**2)
                #P_t = (p_pion+p_proton)**2
                P_t = (batch['REC::Particle_px'][i][kk]+batch['REC::Particle_px'][i][j])**2+(batch['REC::Particle_py'][i][kk]+batch['REC::Particle_py'][i][j])**2+(batch['REC::Particle_pz'][i][kk]+batch['REC::Particle_pz'][i][j])**2
                fourth.imass = [np.sqrt(E_t-P_t)]   '''


                third+= [fourth] 
'''
step = 45000
for batch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_3.hipo'],banks=["MC::Lund", "REC::Particle", "REC::Traj"],step=step):
    print(batch.keys()) 
    #batch.keys()
    dic = batch.keys()  
    break  
all_keys            = list(batch.keys())
rec_particle_name   = 'REC::Particle'
rec_particle_keys   = get_bank_keys(rec_particle_name,all_keys) 
rec_traj_name       = 'REC::Traj'
rec_traj_keys       = get_bank_keys(rec_traj_name,all_keys)
#rec_kinematics_name = 'REC::Kinematics' 
#rec_kinematics_keys = get_bank_keys(rec_kinematics_name,all_keys)
mc_lund_name        = 'MC::Lund'
mc_lund_keys        = get_bank_keys(mc_lund_name,all_keys) 

# rec_kinematics_keys = ['REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 
#                       'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass']
#fulll = [] 
tst = []
y=[]
kinn = []  
idkkk =[]
for i in range(step):
    # Get REC::Particle bank
    rec_particle_event_table = get_event_table(rec_particle_keys,i,batch,dtype=float)

    # Get REC::Traj bank
    rec_traj_event_table = get_event_table(rec_traj_keys,i,batch,dtype=float)

    # Get REC::Kinematics bank
   # rec_kinematics_event_table = get_event_table(rec_kinematics_keys,i,battch,dtype=float)

    # Get MC::Lund bank and MC->REC matching indices
    mc_lund_event_table = get_event_table(mc_lund_keys,i,batch,dtype=float)
    match_indices  = get_match_indices(rec_particle_event_table,mc_lund_event_table)
    
    decay = [3122,[2212,-211]] 
    has_decay, rec_indices = check_has_decay(rec_particle_event_table,mc_lund_event_table,match_indices,decay,
                                   rec_particle_pid_idx=0,mc_lund_pid_idx=3,mc_lund_parent_idx=4,mc_lund_daughter_idx=5)
    
    #print(rec_indices)
    #getting the index of where the electron proton and pion r 
    savee = batch['REC::Particle_pid'][i]
    j = savee.index(2212); kk = savee.index(-211); elc = savee.index(11) #elc is electron, used for checking that it matches up, j is proton, kk is pionn 
    #ckeck
    #making sure that it is not bad (for the match) #if is bad then lots more stuff to do 
    if match_indices[kk][1]!=-1: 
        if (np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)<1):
            if kk in rec_indices and has_decay == True: 
                track_hit = []
                indexp = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == j]
                indexpi = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == kk]
                #get all the data we need here. 

                totalpos =[ [batch['REC::Traj_x'][i][ii] for ii in indexpi], [ batch['REC::Traj_y'][i][ii] for ii in indexpi], [ batch['REC::Traj_z'][i][ii] for ii in indexpi],
                          [batch['REC::Traj_cx'][i][ii] for ii in indexpi], [ batch['REC::Traj_cy'][i][ii] for ii in indexpi], [ batch['REC::Traj_cz'][i][ii] for ii in indexpi]]


                s = 0
                ss =0 
                for dd in range(len(indexpi)):
                    track_hit += [[batch['REC::Traj_x'][i][indexpi[dd]], batch['REC::Traj_y'][i][indexpi[dd]], batch['REC::Traj_z'][i][indexpi[dd]],
                                 batch['REC::Traj_cx'][i][indexpi[dd]], batch['REC::Traj_cy'][i][indexpi[dd]], batch['REC::Traj_cz'][i][indexpi[dd]],
                                   batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk]]]#,
                                 # batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk],
                                   #random.gauss(batch['REC::Particle_px'][i][kk], 1), random.gauss(batch['REC::Particle_py'][i][kk], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][kk], 1),
                                  # random.gauss(batch['REC::Particle_vx'][i][kk], 3.394), random.gauss(batch['REC::Particle_vy'][i][kk], 3.394), 
                                   #random.gauss(batch['REC::Particle_vz'][i][kk], 3.394)]]
                    s +=1
                #track_hit+=[[batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                 #           batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk]]]
                ss =0
                for dd in range(len(indexp)): 
                    track_hit += [[batch['REC::Traj_x'][i][indexp[dd]], batch['REC::Traj_y'][i][indexp[dd]], batch['REC::Traj_z'][i][indexp[dd]],
                                   batch['REC::Traj_cx'][i][indexp[dd]], batch['REC::Traj_cy'][i][indexp[dd]], batch['REC::Traj_cz'][i][indexp[dd]],
                                  batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j]]]#,
                                  #batch['REC::Particle_vx'][i][j], batch['REC::Particle_vy'][i][j], batch['REC::Particle_vz'][i][j],
                                   #random.gauss(batch['REC::Particle_px'][i][j], 1), random.gauss(batch['REC::Particle_py'][i][j], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][j], 1),
                                  # random.gauss(batch['REC::Particle_vx'][i][j], 3.394), random.gauss(batch['REC::Particle_vy'][i][j], 3.394), 
                                   #random.gauss(batch['REC::Particle_vz'][i][j], 3.394)]]

                    ss+=1 


                y = [batch['MC::Lund_vz'][i][match_indices[kk][1]] ] 
                #y = [np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)]  

                #track_hit += [[np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)]]

                start = []
                end = []
                for gg in range(s-1):
                    start.append(gg)
                    end.append(gg+1)
                for ggg in range(ss-1): 
                    start.append(s+ggg)
                    end.append(s+ggg+1)
                fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
               # fourth.kinematics = [battch['REC::Kinematics_Q2'][i][0], battch['REC::Kinematics_nu'][i][0], battch['REC::Kinematics_W'][i][0], battch['REC::Kinematics_x'][i][0],  
                #                     battch['REC::Kinematics_y'][i][0], battch['REC::Kinematics_z'][i][0], battch['REC::Kinematics_xF'][i][0], battch['REC::Kinematics_mass'][i][0]]
                fourth.rec  = [batch['REC::Particle_vz'][i][kk]] 
                #fourth.mom = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                fourth.elc = [batch['REC::Particle_vz'][i][0] ] 
            #fourth.rec = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                #idkkk+=[[batch['REC::Particle_vz'][i][kk], batch['REC::Particle_vz'][i][j]]]
                #invarient masss#######################################################################################################
                fourth.imass = [batch['REC::Kinematics_mass'][i][0]]
                third+= [fourth] '''


step = 45000
for batch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_4.hipo'],banks=["MC::Lund", "REC::Particle", "REC::Traj"],step=step):
    print(batch.keys()) 
    #batch.keys()
    dic = batch.keys() 
    break  
all_keys            = list(batch.keys())
rec_particle_name   = 'REC::Particle'
rec_particle_keys   = get_bank_keys(rec_particle_name,all_keys) 
rec_traj_name       = 'REC::Traj'
rec_traj_keys       = get_bank_keys(rec_traj_name,all_keys)
#rec_kinematics_name = 'REC::Kinematics' 
#rec_kinematics_keys = get_bank_keys(rec_kinematics_name,all_keys)
mc_lund_name        = 'MC::Lund'
mc_lund_keys        = get_bank_keys(mc_lund_name,all_keys) 

# rec_kinematics_keys = ['REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 
#                       'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass']
#fulll = [] 
tst = []
y=[]
kinn = []  
idkkk =[]
for i in range(step):
    # Get REC::Particle bank
    rec_particle_event_table = get_event_table(rec_particle_keys,i,batch,dtype=float)

    # Get REC::Traj bank
    rec_traj_event_table = get_event_table(rec_traj_keys,i,batch,dtype=float)

    # Get REC::Kinematics bank
   # rec_kinematics_event_table = get_event_table(rec_kinematics_keys,i,battch,dtype=float)

    # Get MC::Lund bank and MC->REC matching indices
    mc_lund_event_table = get_event_table(mc_lund_keys,i,batch,dtype=float)
    match_indices  = get_match_indices(rec_particle_event_table,mc_lund_event_table)
    
    decay = [3122,[2212,-211]] 
    has_decay, rec_indices = check_has_decay(rec_particle_event_table,mc_lund_event_table,match_indices,decay,
                                   rec_particle_pid_idx=0,mc_lund_pid_idx=3,mc_lund_parent_idx=4,mc_lund_daughter_idx=5)
    
    #print(rec_indices)
    #getting the index of where the electron proton and pion r 
    savee = batch['REC::Particle_pid'][i]
    j = savee.index(2212); kk = savee.index(-211); elc = savee.index(11) #elc is electron, used for checking that it matches up, j is proton, kk is pionn 
    #ckeck
    #making sure that it is not bad (for the match) #if is bad then lots more stuff to do 
    if match_indices[kk][1]!=-1: 
        if (np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)<1):
            if kk in rec_indices and has_decay == True: 
                track_hit = []
                indexp = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == j]
                indexpi = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == kk]
                #get all the data we need here. 

                totalpos =[ [batch['REC::Traj_x'][i][ii] for ii in indexpi], [ batch['REC::Traj_y'][i][ii] for ii in indexpi], [ batch['REC::Traj_z'][i][ii] for ii in indexpi],
                          [batch['REC::Traj_cx'][i][ii] for ii in indexpi], [ batch['REC::Traj_cy'][i][ii] for ii in indexpi], [ batch['REC::Traj_cz'][i][ii] for ii in indexpi]]


                s = 0
                ss =0 
                for dd in range(len(indexpi)):
                    track_hit += [[batch['REC::Traj_x'][i][indexpi[dd]], batch['REC::Traj_y'][i][indexpi[dd]], batch['REC::Traj_z'][i][indexpi[dd]],
                                 batch['REC::Traj_cx'][i][indexpi[dd]], batch['REC::Traj_cy'][i][indexpi[dd]], batch['REC::Traj_cz'][i][indexpi[dd]],
                                   batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk]]]#,
                                 # batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk],
                                   #random.gauss(batch['REC::Particle_px'][i][kk], 1), random.gauss(batch['REC::Particle_py'][i][kk], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][kk], 1),
                                  # random.gauss(batch['REC::Particle_vx'][i][kk], 3.394), random.gauss(batch['REC::Particle_vy'][i][kk], 3.394), 
                                   #random.gauss(batch['REC::Particle_vz'][i][kk], 3.394)]]
                    s +=1
                #track_hit+=[[batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                 #           batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk]]]
                ss =0
                for dd in range(len(indexp)): 
                    track_hit += [[batch['REC::Traj_x'][i][indexp[dd]], batch['REC::Traj_y'][i][indexp[dd]], batch['REC::Traj_z'][i][indexp[dd]],
                                   batch['REC::Traj_cx'][i][indexp[dd]], batch['REC::Traj_cy'][i][indexp[dd]], batch['REC::Traj_cz'][i][indexp[dd]],
                                  batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j]]]#,
                                  #batch['REC::Particle_vx'][i][j], batch['REC::Particle_vy'][i][j], batch['REC::Particle_vz'][i][j],
                                   #random.gauss(batch['REC::Particle_px'][i][j], 1), random.gauss(batch['REC::Particle_py'][i][j], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][j], 1),
                                  # random.gauss(batch['REC::Particle_vx'][i][j], 3.394), random.gauss(batch['REC::Particle_vy'][i][j], 3.394), 
                                   #random.gauss(batch['REC::Particle_vz'][i][j], 3.394)]]

                    ss+=1 


                y = [batch['MC::Lund_vz'][i][match_indices[kk][1]] ] 
                #y = [np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)]  

                #track_hit += [[np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)]]

                start = []
                end = []
                for gg in range(s-1):
                    start.append(gg)
                    end.append(gg+1)
                for ggg in range(ss-1): 
                    start.append(s+ggg)
                    end.append(s+ggg+1)
                fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
               # fourth.kinematics = [battch['REC::Kinematics_Q2'][i][0], battch['REC::Kinematics_nu'][i][0], battch['REC::Kinematics_W'][i][0], battch['REC::Kinematics_x'][i][0],  
                #                     battch['REC::Kinematics_y'][i][0], battch['REC::Kinematics_z'][i][0], battch['REC::Kinematics_xF'][i][0], battch['REC::Kinematics_mass'][i][0]]
                fourth.rec  = [batch['REC::Particle_vz'][i][kk]] 
                #fourth.mom = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                fourth.elc = [batch['REC::Particle_vz'][i][0] ] 
            #fourth.rec = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                #idkkk+=[[batch['REC::Particle_vz'][i][kk], batch['REC::Particle_vz'][i][j]]]
                #invarient masss#######################################################################################################
                fourth.imass = [batch['REC::Kinematics_mass'][i][0]]
                third+= [fourth]  
                
data_list = third 
root = '/work/clas12/users/mencke/home_L_1'     
class MyDataset(InMemoryDataset):
    def __init__(self, root, data_list, transform=None): 
        self.data_list = data_list
        super().__init__(root, transform) 
        self.load(self.processed_paths[0])

    @property
    def processed_file_names(self):
        return 'data.pt'
 
    def process(self):
        self.save(self.data_list, self.processed_paths[0])    

MyDataset(root, third)        

dict_keys(['MC::Lund_index', 'MC::Lund_lifetime', 'MC::Lund_type', 'MC::Lund_pid', 'MC::Lund_parent', 'MC::Lund_daughter', 'MC::Lund_px', 'MC::Lund_py', 'MC::Lund_pz', 'MC::Lund_energy', 'MC::Lund_mass', 'MC::Lund_vx', 'MC::Lund_vy', 'MC::Lund_vz', 'REC::Particle_pid', 'REC::Particle_px', 'REC::Particle_py', 'REC::Particle_pz', 'REC::Particle_vx', 'REC::Particle_vy', 'REC::Particle_vz', 'REC::Particle_vt', 'REC::Particle_charge', 'REC::Particle_beta', 'REC::Particle_chi2pid', 'REC::Particle_status', 'REC::Traj_pindex', 'REC::Traj_index', 'REC::Traj_detector', 'REC::Traj_layer', 'REC::Traj_x', 'REC::Traj_y', 'REC::Traj_z', 'REC::Traj_cx', 'REC::Traj_cy', 'REC::Traj_cz', 'REC::Traj_path', 'REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass'])
dict_keys(['MC::Lund_index', 'MC::Lund_lifetime', 'MC::L

KeyError: 'REC::Kinematics_mass'

In [6]:
#mkaing half and half data sets, half is lambda combinning two whole sets, third is 10K of it without the lambda force
#modified now to be quaoter datasets. 
from torch_geometric.data import Dataset, download_url, InMemoryDataset

 #third is list of graphss   
third = []
kkkk = 0
#for j in range(1):     
 #   number = 5
    #if j==2:
      #  number = 2
j=0  
number = 2 

for ii in range( number): 
    iii=  ii +3
    if j==0:
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_'+str(iii)+'.hipo'
    if j==1:
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3316_'+str(iii)+'.hipo'
    if j==2:
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3317_'+str(iii)+'.hipo'
    if j==3 :
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3318_'+str(iii)+'.hipo'
    if j==4:
        filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3327_'+str(iii)+'.hipo'

    step = 40000 
    for batch in hp.iterate([filee],banks=["MC::Lund", "REC::Particle", "REC::Traj", "REC::Kinematics"],step=step): 
       # print(batch.keys()) 
        #batch.keys()
        dic = batch.keys() 
        break   

    #for battch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_0.hipo'],banks=["REC::Kinematics"],step=step):
        #print(battch.keys()) 
        #batch.keys()
        #dic = battch.keys() 
       # break  
    #
    all_keys            = list(batch.keys())
    rec_particle_name   = 'REC::Particle'
    rec_particle_keys   = get_bank_keys(rec_particle_name,all_keys)
    rec_traj_name       = 'REC::Traj'
    rec_traj_keys       = get_bank_keys(rec_traj_name,all_keys)
    rec_kinematics_name = 'REC::Kinematics' 
    rec_kinematics_keys = get_bank_keys(rec_kinematics_name,all_keys)
    mc_lund_name        = 'MC::Lund'
    mc_lund_keys        = get_bank_keys(mc_lund_name,all_keys) 

    # rec_kinematics_keys = ['REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 
    #                       'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass']
    #fulll = [] 
    tst = []
    y=[]


    #j is proton, kk is pion


    kinn = []  
    idkkk =[]
    for i in range(step):
        # Get REC::Particle bank
        rec_particle_event_table = get_event_table(rec_particle_keys,i,batch,dtype=float)

        # Get REC::Traj bank
        rec_traj_event_table = get_event_table(rec_traj_keys,i,batch,dtype=float)

        # Get REC::Kinematics bank
        rec_kinematics_event_table = get_event_table(rec_kinematics_keys,i,batch,dtype=float)

        # Get MC::Lund bank and MC->REC matching indices
        mc_lund_event_table = get_event_table(mc_lund_keys,i,batch,dtype=float)
        match_indices  = get_match_indices(rec_particle_event_table,mc_lund_event_table)

        decay = [3122,[2212,-211]] 
        has_decay, rec_indices = check_has_decay(rec_particle_event_table,mc_lund_event_table,match_indices,decay,
                                       rec_particle_pid_idx=0,mc_lund_pid_idx=3,mc_lund_parent_idx=4,mc_lund_daughter_idx=5)

        #print(rec_indices)
        #getting the index of where the electron proton and pion r 
        savee = batch['REC::Particle_pid'][i]
        j = savee.index(2212); kk = savee.index(-211); elc = savee.index(11) #elc is electron, used for checking that it matches up, j is proton, kk is pionn 
        #ckeck
        #making sure that it is not bad (for the match) #if is bad then lots more stuff to do 
        if match_indices[kk][1]!=-1: 
            #if (np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)<1):
            if kk in rec_indices and has_decay == True:    
                track_hit = []
                indexp = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == j]
                indexpi = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == kk]
                #get all the data we need here. 

                totalpos =[ [batch['REC::Traj_x'][i][ii] for ii in indexpi], [ batch['REC::Traj_y'][i][ii] for ii in indexpi], [ batch['REC::Traj_z'][i][ii] for ii in indexpi],
                          [batch['REC::Traj_cx'][i][ii] for ii in indexpi], [ batch['REC::Traj_cy'][i][ii] for ii in indexpi], [ batch['REC::Traj_cz'][i][ii] for ii in indexpi]]


                s = 0
                ss =0 
                for dd in range(len(indexpi)):
                    track_hit += [[batch['REC::Traj_x'][i][indexpi[dd]], batch['REC::Traj_y'][i][indexpi[dd]], batch['REC::Traj_z'][i][indexpi[dd]],
                                 batch['REC::Traj_cx'][i][indexpi[dd]], batch['REC::Traj_cy'][i][indexpi[dd]], batch['REC::Traj_cz'][i][indexpi[dd]],
                                   batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk]]]
                                 # batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk],
                                   #random.gauss(batch['REC::Particle_px'][i][kk], 1), random.gauss(batch['REC::Particle_py'][i][kk], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][kk], 1),
                                  # random.gauss(batch['REC::Particle_vx'][i][kk], 3.394), random.gauss(batch['REC::Particle_vy'][i][kk], 3.394), 
                                  # random.gauss(batch['REC::Particle_vz'][i][kk], 3.394)]]
                    s +=1
                #track_hit+=[[batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                 #           batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk]]]
                ss =0 
                for dd in range(len(indexp)): 
                    track_hit += [[batch['REC::Traj_x'][i][indexp[dd]], batch['REC::Traj_y'][i][indexp[dd]], batch['REC::Traj_z'][i][indexp[dd]],
                                   batch['REC::Traj_cx'][i][indexp[dd]], batch['REC::Traj_cy'][i][indexp[dd]], batch['REC::Traj_cz'][i][indexp[dd]],
                                  batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j]]]
                                  #batch['REC::Particle_vx'][i][j], batch['REC::Particle_vy'][i][j], batch['REC::Particle_vz'][i][j],
                                   #random.gauss(batch['REC::Particle_px'][i][j], 1), random.gauss(batch['REC::Particle_py'][i][j], 1), 
                                   #random.gauss(batch['REC::Particle_pz'][i][j], 1),
                                  # random.gauss(batch['REC::Particle_vx'][i][j], 3.394), random.gauss(batch['REC::Particle_vy'][i][j], 3.394), 
                                  # random.gauss(batch['REC::Particle_vz'][i][j], 3.394)]]

                    ss+=1  


                y = [batch['MC::Lund_vz'][i][match_indices[kk][1]] ] 
                #y = [np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)]  

                #track_hit += [[np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)]]

                start = []
                end = []
                for gg in range(s-1):
                    start.append(gg)
                    end.append(gg+1)
                for ggg in range(ss-1): 
                    start.append(s+ggg) 
                    end.append(s+ggg+1)
                #fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
                fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
               # fourth.kinematics = [battch['REC::Kinematics_Q2'][i][0], battch['REC::Kinematics_nu'][i][0], battch['REC::Kinematics_W'][i][0], battch['REC::Kinematics_x'][i][0],  
                #                     battch['REC::Kinematics_y'][i][0], battch['REC::Kinematics_z'][i][0], battch['REC::Kinematics_xF'][i][0], battch['REC::Kinematics_mass'][i][0]]
                fourth.rec  = [batch['REC::Particle_vz'][i][kk]] 
                #fourth.mom = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                fourth.elc = [batch['REC::Particle_vz'][i][0] ] 
                #fourth.rec = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
                #idkkk+=[[batch['REC::Particle_vz'][i][kk], batch['REC::Particle_vz'][i][j]]]
                #invarient masss#######################################################################################################
                fourth.imass = [batch['REC::Kinematics_mass'][i][0]] 
                if kk in rec_indices and has_decay == True:
                    fourth.LamT = [1]
                else :
                    fourth.LamT = [0]
                third+= [fourth]
    kkkk+=1
    print(kkkk) 
ii=3 
j=1
if j==0:
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_'+str(ii)+'.hipo'
if j==1:
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3316_'+str(ii)+'.hipo'
if j==2:
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3317_'+str(ii)+'.hipo'
if j==3 :
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3318_'+str(ii)+'.hipo'
if j==4:
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3327_'+str(ii)+'.hipo'

step = 12000   
for batch in hp.iterate([filee],banks=["MC::Lund", "REC::Particle", "REC::Traj", "REC::Kinematics"],step=step): 
   # print(batch.keys()) 
    #batch.keys()
    dic = batch.keys()  
    break   

#for battch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_0.hipo'],banks=["REC::Kinematics"],step=step):
    #print(battch.keys()) 
    #batch.keys()
    #dic = battch.keys() 
   # break  
#
all_keys            = list(batch.keys())
rec_particle_name   = 'REC::Particle'
rec_particle_keys   = get_bank_keys(rec_particle_name,all_keys)
rec_traj_name       = 'REC::Traj'
rec_traj_keys       = get_bank_keys(rec_traj_name,all_keys)
rec_kinematics_name = 'REC::Kinematics' 
rec_kinematics_keys = get_bank_keys(rec_kinematics_name,all_keys)
mc_lund_name        = 'MC::Lund'
mc_lund_keys        = get_bank_keys(mc_lund_name,all_keys) 

# rec_kinematics_keys = ['REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 
#                       'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass']
#fulll = [] 
tst = []
y=[]


#j is proton, kk is pion


kinn = []  
idkkk =[]
for i in range(step):
    # Get REC::Particle bank
    rec_particle_event_table = get_event_table(rec_particle_keys,i,batch,dtype=float)

    # Get REC::Traj bank
    rec_traj_event_table = get_event_table(rec_traj_keys,i,batch,dtype=float)

    # Get REC::Kinematics bank
    rec_kinematics_event_table = get_event_table(rec_kinematics_keys,i,batch,dtype=float)

    # Get MC::Lund bank and MC->REC matching indices
    mc_lund_event_table = get_event_table(mc_lund_keys,i,batch,dtype=float)
    match_indices  = get_match_indices(rec_particle_event_table,mc_lund_event_table)

    decay = [3122,[2212,-211]] 
    has_decay, rec_indices = check_has_decay(rec_particle_event_table,mc_lund_event_table,match_indices,decay,
                                   rec_particle_pid_idx=0,mc_lund_pid_idx=3,mc_lund_parent_idx=4,mc_lund_daughter_idx=5)

    #print(rec_indices)
    #getting the index of where the electron proton and pion r 
    savee = batch['REC::Particle_pid'][i]
    j = savee.index(2212); kk = savee.index(-211); elc = savee.index(11) #elc is electron, used for checking that it matches up, j is proton, kk is pionn 
    #ckeck
    #making sure that it is not bad (for the match) #if is bad then lots more stuff to do 
    if match_indices[kk][1]!=-1: 
        #if (np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)<1):
        #if kk in rec_indices and has_decay == True:     
        track_hit = []
        indexp = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == j]
        indexpi = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == kk]
        #get all the data we need here. 

        totalpos =[ [batch['REC::Traj_x'][i][ii] for ii in indexpi], [ batch['REC::Traj_y'][i][ii] for ii in indexpi], [ batch['REC::Traj_z'][i][ii] for ii in indexpi],
                  [batch['REC::Traj_cx'][i][ii] for ii in indexpi], [ batch['REC::Traj_cy'][i][ii] for ii in indexpi], [ batch['REC::Traj_cz'][i][ii] for ii in indexpi]]


        s = 0
        ss =0 
        for dd in range(len(indexpi)):
            track_hit += [[batch['REC::Traj_x'][i][indexpi[dd]], batch['REC::Traj_y'][i][indexpi[dd]], batch['REC::Traj_z'][i][indexpi[dd]],
                         batch['REC::Traj_cx'][i][indexpi[dd]], batch['REC::Traj_cy'][i][indexpi[dd]], batch['REC::Traj_cz'][i][indexpi[dd]],
                           batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk]]]
                         # batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk],
                           #random.gauss(batch['REC::Particle_px'][i][kk], 1), random.gauss(batch['REC::Particle_py'][i][kk], 1), 
                           #random.gauss(batch['REC::Particle_pz'][i][kk], 1),
                          # random.gauss(batch['REC::Particle_vx'][i][kk], 3.394), random.gauss(batch['REC::Particle_vy'][i][kk], 3.394), 
                          # random.gauss(batch['REC::Particle_vz'][i][kk], 3.394)]]
            s +=1
        #track_hit+=[[batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
         #           batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk]]]
        ss =0 
        for dd in range(len(indexp)): 
            track_hit += [[batch['REC::Traj_x'][i][indexp[dd]], batch['REC::Traj_y'][i][indexp[dd]], batch['REC::Traj_z'][i][indexp[dd]],
                           batch['REC::Traj_cx'][i][indexp[dd]], batch['REC::Traj_cy'][i][indexp[dd]], batch['REC::Traj_cz'][i][indexp[dd]],
                          batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j]]]
                          #batch['REC::Particle_vx'][i][j], batch['REC::Particle_vy'][i][j], batch['REC::Particle_vz'][i][j],
                           #random.gauss(batch['REC::Particle_px'][i][j], 1), random.gauss(batch['REC::Particle_py'][i][j], 1), 
                           #random.gauss(batch['REC::Particle_pz'][i][j], 1),
                           #random.gauss(batch['REC::Particle_vx'][i][j], 3.394), random.gauss(batch['REC::Particle_vy'][i][j], 3.394), 
                          # random.gauss(batch['REC::Particle_vz'][i][j], 3.394)]]

            ss+=1  


        y = [batch['MC::Lund_vz'][i][match_indices[kk][1]] ] 
        #y = [np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)]  

        #track_hit += [[np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)]]

        start = []
        end = []
        for gg in range(s-1):
            start.append(gg)
            end.append(gg+1)
        for ggg in range(ss-1): 
            start.append(s+ggg) 
            end.append(s+ggg+1)
        #fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
        fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
       # fourth.kinematics = [battch['REC::Kinematics_Q2'][i][0], battch['REC::Kinematics_nu'][i][0], battch['REC::Kinematics_W'][i][0], battch['REC::Kinematics_x'][i][0],  
        #                     battch['REC::Kinematics_y'][i][0], battch['REC::Kinematics_z'][i][0], battch['REC::Kinematics_xF'][i][0], battch['REC::Kinematics_mass'][i][0]]
        fourth.rec  = [batch['REC::Particle_vz'][i][kk]] 
        #fourth.mom = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
        fourth.elc = [batch['REC::Particle_vz'][i][0] ] 
        #fourth.rec = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
        #idkkk+=[[batch['REC::Particle_vz'][i][kk], batch['REC::Particle_vz'][i][j]]]
        #invarient masss#######################################################################################################
        fourth.imass = [batch['REC::Kinematics_mass'][i][0]] 
        if kk in rec_indices and has_decay == True:
            fourth.LamT = [1]
        else :
            fourth.LamT = [0]
        third+= [fourth]
kkkk+=1
print(kkkk) 

data_list = third  
root = '/work/clas12/users/mencke/HalfL_sixth'           
class MyDataset(InMemoryDataset):
    def __init__(self, root, data_list, transform=None): 
        self.data_list = data_list
        super().__init__(root, transform)  
        self.load(self.processed_paths[0])

    @property
    def processed_file_names(self): 
        return 'data.pt' 
 
    def process(self):
        self.save(self.data_list, self.processed_paths[0])      

MyDataset(root, third)      

ValueError: HipoFileIterator: length of filenames must be > 0

In [13]:
j=4  
number = 2 

#for ii in range( number): 
iii=  ii +3
iii =4 8
if j==0:
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_'+str(iii)+'.hipo'
if j==1:
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3316_'+str(iii)+'.hipo'
if j==2:
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3317_'+str(iii)+'.hipo'
if j==3 :
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3318_'+str(iii)+'.hipo'
if j==4:
    filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3327_'+str(iii)+'.hipo'
filee = '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_1.hipo' 

step = 40000 
for batch in hp.iterate([filee],banks=["MC::Lund", "REC::Particle", "REC::Traj", "REC::Kinematics"],step=step): 
   # print(batch.keys()) 
    #batch.keys()
    dic = batch.keys() 
    break

ValueError: HipoFileIterator: length of filenames must be > 0

In [12]:
from torch_geometric.data import Dataset, download_url, InMemoryDataset   

step = 20000
for batch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_1.hipo'],banks=["MC::Lund", "REC::Particle", "REC::Traj", "REC::Kinematics"],step=step):
    print(batch.keys()) 
    #batch.keys()
    dic = batch.keys() 
    break  
    
#for battch in hp.iterate(['/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_2_12_24/analysis2/kinematics_out_skim_50nA_OB_job_3313_0.hipo'],banks=["REC::Kinematics"],step=step):
    #print(battch.keys()) 
    #batch.keys()
    #dic = battch.keys() 
   # break  
#

all_keys            = list(batch.keys())
rec_particle_name   = 'REC::Particle'
rec_particle_keys   = get_bank_keys(rec_particle_name,all_keys)
rec_traj_name       = 'REC::Traj'
rec_traj_keys       = get_bank_keys(rec_traj_name,all_keys)
rec_kinematics_name = 'REC::Kinematics' 
rec_kinematics_keys = get_bank_keys(rec_kinematics_name,all_keys)
mc_lund_name        = 'MC::Lund'
mc_lund_keys        = get_bank_keys(mc_lund_name,all_keys) 

# rec_kinematics_keys = ['REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 
#                       'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass']
#fulll = [] 
tst = []
y=[]
#third is list of graphss 
third = []
#j is proton, kk is pion


kinn = []  
idkkk =[]
for i in range(step):
    # Get REC::Particle bank
    rec_particle_event_table = get_event_table(rec_particle_keys,i,batch,dtype=float)

    # Get REC::Traj bank
    rec_traj_event_table = get_event_table(rec_traj_keys,i,batch,dtype=float)

    # Get REC::Kinematics bank
    rec_kinematics_event_table = get_event_table(rec_kinematics_keys,i,batch,dtype=float)

    # Get MC::Lund bank and MC->REC matching indices
    mc_lund_event_table = get_event_table(mc_lund_keys,i,batch,dtype=float)
    match_indices  = get_match_indices(rec_particle_event_table,mc_lund_event_table)
    
    decay = [3122,[2212,-211]] 
    has_decay, rec_indices = check_has_decay(rec_particle_event_table,mc_lund_event_table,match_indices,decay,
                                   rec_particle_pid_idx=0,mc_lund_pid_idx=3,mc_lund_parent_idx=4,mc_lund_daughter_idx=5)
    
    #print(rec_indices)
    #getting the index of where the electron proton and pion r 
    savee = batch['REC::Particle_pid'][i]
    j = savee.index(2212); kk = savee.index(-211); elc = savee.index(11) #elc is electron, used for checking that it matches up, j is proton, kk is pionn 
    #ckeck
    #making sure that it is not bad (for the match) #if is bad then lots more stuff to do 
    if match_indices[kk][1]!=-1: 
        #if (np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)<1):
        #if kk in rec_indices and has_decay == True: 
        track_hit = []
        indexp = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == j]
        indexpi = [index for (index, item) in enumerate(batch['REC::Traj_pindex'][i]) if item == kk]
        #get all the data we need here. 

        totalpos =[ [batch['REC::Traj_x'][i][ii] for ii in indexpi], [ batch['REC::Traj_y'][i][ii] for ii in indexpi], [ batch['REC::Traj_z'][i][ii] for ii in indexpi],
                  [batch['REC::Traj_cx'][i][ii] for ii in indexpi], [ batch['REC::Traj_cy'][i][ii] for ii in indexpi], [ batch['REC::Traj_cz'][i][ii] for ii in indexpi]]


        s = 0
        ss =0 
        for dd in range(len(indexpi)):
            track_hit += [[batch['REC::Traj_x'][i][indexpi[dd]], batch['REC::Traj_y'][i][indexpi[dd]], batch['REC::Traj_z'][i][indexpi[dd]],
                         batch['REC::Traj_cx'][i][indexpi[dd]], batch['REC::Traj_cy'][i][indexpi[dd]], batch['REC::Traj_cz'][i][indexpi[dd]],
                           batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
                         # batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk],
                           #random.gauss(batch['REC::Particle_px'][i][kk], 1), random.gauss(batch['REC::Particle_py'][i][kk], 1), 
                           #random.gauss(batch['REC::Particle_pz'][i][kk], 1),
                           random.gauss(batch['REC::Particle_vx'][i][kk], 3.394), random.gauss(batch['REC::Particle_vy'][i][kk], 3.394), 
                           random.gauss(batch['REC::Particle_vz'][i][kk], 3.394)]]
            s +=1
        #track_hit+=[[batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk],
         #           batch['REC::Particle_vx'][i][kk], batch['REC::Particle_vy'][i][kk], batch['REC::Particle_vz'][i][kk]]]
        ss =0
        for dd in range(len(indexp)): 
            track_hit += [[batch['REC::Traj_x'][i][indexp[dd]], batch['REC::Traj_y'][i][indexp[dd]], batch['REC::Traj_z'][i][indexp[dd]],
                           batch['REC::Traj_cx'][i][indexp[dd]], batch['REC::Traj_cy'][i][indexp[dd]], batch['REC::Traj_cz'][i][indexp[dd]],
                          batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j],
                          #batch['REC::Particle_vx'][i][j], batch['REC::Particle_vy'][i][j], batch['REC::Particle_vz'][i][j],
                           #random.gauss(batch['REC::Particle_px'][i][j], 1), random.gauss(batch['REC::Particle_py'][i][j], 1), 
                           #random.gauss(batch['REC::Particle_pz'][i][j], 1),
                           random.gauss(batch['REC::Particle_vx'][i][j], 3.394), random.gauss(batch['REC::Particle_vy'][i][j], 3.394), 
                           random.gauss(batch['REC::Particle_vz'][i][j], 3.394)]]

            ss+=1  


        y = [batch['MC::Lund_vz'][i][match_indices[kk][1]] ] 
        #y = [np.sqrt(batch['MC::Lund_px'][i][match_indices[kk][1]]**2+batch['MC::Lund_py'][i][match_indices[kk][1]]**2+batch['MC::Lund_pz'][i][match_indices[kk][1]]**2)]  

        #track_hit += [[np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)]]

        start = []
        end = []
        for gg in range(s-1):
            start.append(gg)
            end.append(gg+1)
        for ggg in range(ss-1): 
            start.append(s+ggg)
            end.append(s+ggg+1)
        #fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
        fourth = dt(x=torch.tensor(track_hit), edge_index = torch.tensor([start, end]), y = torch.tensor(y))
       # fourth.kinematics = [battch['REC::Kinematics_Q2'][i][0], battch['REC::Kinematics_nu'][i][0], battch['REC::Kinematics_W'][i][0], battch['REC::Kinematics_x'][i][0],  
        #                     battch['REC::Kinematics_y'][i][0], battch['REC::Kinematics_z'][i][0], battch['REC::Kinematics_xF'][i][0], battch['REC::Kinematics_mass'][i][0]]
        fourth.rec  = [batch['REC::Particle_vz'][i][kk]] 
        #fourth.mom = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
        fourth.elc = [batch['REC::Particle_vz'][i][0] ] 
        #fourth.rec = [np.sqrt(batch['REC::Particle_px'][i][kk]**2+batch['REC::Particle_py'][i][kk]**2+batch['REC::Particle_pz'][i][kk]**2)]
        #idkkk+=[[batch['REC::Particle_vz'][i][kk], batch['REC::Particle_vz'][i][j]]]
        #invarient masss#######################################################################################################
        fourth.imass = [batch['REC::Kinematics_mass'][i][0]] 

        '''
        p_pion = np.array([batch['REC::Particle_px'][i][kk], batch['REC::Particle_py'][i][kk], batch['REC::Particle_pz'][i][kk]]) 
        p_proton = np.array([batch['REC::Particle_px'][i][j], batch['REC::Particle_py'][i][j], batch['REC::Particle_pz'][i][j]])
        P_mass = 938 *10**-3 #GeV 
        pi_mass = 139 *10**-3 #GeV 
        E_pion = np.sqrt(pi_mass**2+np.dot(p_pion, p_pion))
        E_proton = np.sqrt(P_mass**2 +np.dot(p_proton, p_proton))
        E_t = (E_pion+E_proton)**2
        P_t = np.dot(p_pion, p_proton)
        fourth.imass = [np.sqrt(E_t-P_t)] '''


        '''
        p_pion = np.sqrt(batch['REC::Particle_px'][i][kk]**2+ batch['REC::Particle_py'][i][kk]**2+ batch['REC::Particle_pz'][i][kk]**2)
        p_proton = np.sqrt(batch['REC::Particle_px'][i][j]**2+ batch['REC::Particle_py'][i][j]**2+ batch['REC::Particle_pz'][i][j]**2)
        P_mass = 938 *10**-3 #GeV 
        pi_mass = 139 *10**-3 #GeV  
        E_pion = np.sqrt(pi_mass**2+p_pion**2)
        E_proton  = np.sqrt(P_mass**2 +p_proton **2)
        #E_t = (E_pion+E_proton)**2
        E_t = (pi_mass+P_mass+p_pion+p_proton)**2 
        #E_t = (E_pion**2+E_proton**2)
        #P_t = (p_pion+p_proton)**2
        P_t = (batch['REC::Particle_px'][i][kk]+batch['REC::Particle_px'][i][j])**2+(batch['REC::Particle_py'][i][kk]+batch['REC::Particle_py'][i][j])**2+(batch['REC::Particle_pz'][i][kk]+batch['REC::Particle_pz'][i][j])**2
        fourth.imass = [np.sqrt(E_t-P_t)]   '''

        third+= [fourth]  

data_list = third 
root = '/work/clas12/users/mencke/L_imass9'     
class MyDataset(InMemoryDataset):
    def __init__(self, root, data_list, transform=None): 
        self.data_list = data_list
        super().__init__(root, transform) 
        self.load(self.processed_paths[0])

    @property
    def processed_file_names(self):
        return 'data.pt'
  

    def process(self):
        self.save(self.data_list, self.processed_paths[0])    

MyDataset(root, third)     

dict_keys(['MC::Lund_index', 'MC::Lund_lifetime', 'MC::Lund_type', 'MC::Lund_pid', 'MC::Lund_parent', 'MC::Lund_daughter', 'MC::Lund_px', 'MC::Lund_py', 'MC::Lund_pz', 'MC::Lund_energy', 'MC::Lund_mass', 'MC::Lund_vx', 'MC::Lund_vy', 'MC::Lund_vz', 'REC::Particle_pid', 'REC::Particle_px', 'REC::Particle_py', 'REC::Particle_pz', 'REC::Particle_vx', 'REC::Particle_vy', 'REC::Particle_vz', 'REC::Particle_vt', 'REC::Particle_charge', 'REC::Particle_beta', 'REC::Particle_chi2pid', 'REC::Particle_status', 'REC::Traj_pindex', 'REC::Traj_index', 'REC::Traj_detector', 'REC::Traj_layer', 'REC::Traj_x', 'REC::Traj_y', 'REC::Traj_z', 'REC::Traj_cx', 'REC::Traj_cy', 'REC::Traj_cz', 'REC::Traj_path', 'REC::Kinematics_idxe', 'REC::Kinematics_idxp', 'REC::Kinematics_idxpi', 'REC::Kinematics_Q2', 'REC::Kinematics_nu', 'REC::Kinematics_W', 'REC::Kinematics_x', 'REC::Kinematics_y', 'REC::Kinematics_z', 'REC::Kinematics_xF', 'REC::Kinematics_mass'])


Processing...
Done!


MyDataset(18464)

In [ ]:
data_list  

In [ ]:
ork/clas12/users/mencke/test_L_immass'  this is without the rec vertex info. trying to get invarient mass. 